

```
# Isto está formatado como código
```



##Inserindo a biblioteca Kaggle




In [ ]:
!pip install kaggle

##Configurando a API do Kaggle

In [ ]:
from google.colab import files
import os

# Crie o conteúdo do kaggle.json
kaggle_json_content = """
{
  "username": "palomaaugustarossi",
  "key": "9cf4bdd7a37681b50ed2306db9ebe62"
}
"""

# Salve o arquivo temporariamente
with open('kaggle.json', 'w') as f:
    f.write(kaggle_json_content)

# Mova para o local correto e ajuste as permissões
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Verifique se está correto
!cat ~/.kaggle/kaggle.json

##Baixando o Dataset e Instalar Dependência

In [ ]:
# Baixar o dataset
!kaggle datasets download -d olistbr/brazilian-ecommerce

# Descompactar os arquivos
!unzip -q brazilian-ecommerce.zip -d data

# Listar arquivos baixados
!ls data

#Instalar PandaSQL
!pip install pandasql

##Configurações do ambiente

Inserção das bibiotecas e padrões de estilo e medidas.

In [ ]:
import pandas as pd
import numpy as np
import math
import os
import glob
import json
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.gridspec as gridspec
import matplotlib.ticker as mtick
from datetime import datetime

from graphviz import Digraph
from IPython.display import display

import folium
from folium.plugins import HeatMap

import ipywidgets as widgets
from IPython.display import display, HTML, clear_output as clear_display_output

from pandasql import sqldf
from math import radians, sin, cos, sqrt, atan2

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.cluster import KMeans


def pysqldf(q):
    return sqldf(q, globals())

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
sns.set_style('whitegrid')

print("Executado")

##Carregar datasets
Forem inseridos os datasets, carregados e no print conseguimos ter uma visão geral de cada um deles atravez das colunas e primeiras linhas de cada um.


In [ ]:
def carregar_datasets():
    print("Carregando datasets da pasta 'data/'...")
    datasets_local = {
        "customers": pd.read_csv("data/olist_customers_dataset.csv"),
        "geolocation": pd.read_csv("data/olist_geolocation_dataset.csv"),
        "order_items": pd.read_csv("data/olist_order_items_dataset.csv"),
        "order_payments": pd.read_csv("data/olist_order_payments_dataset.csv"),
        "order_reviews": pd.read_csv("data/olist_order_reviews_dataset.csv"),
        "orders": pd.read_csv("data/olist_orders_dataset.csv"),
        "products": pd.read_csv("data/olist_products_dataset.csv"),
        "sellers": pd.read_csv("data/olist_sellers_dataset.csv"),
        "product_category_name_translation": pd.read_csv("data/product_category_name_translation.csv")
    }
    for name, df in datasets_local.items():
        globals()[name] = df
    print("Datasets carregados e disponíveis globalmente para pandasql.")

    print("\n--- Amostra dos Datasets Carregados (Primeiras 5 Linhas) ---")
    for name, df in datasets_local.items():
        print(f"\nDataset: {name}")
        print(df.head())
        print("--------------------")

# Chamar a função para carregar os datasets
carregar_datasets()

print("\nExecutado")

# **Limpeza dos dados**

##Tratamento dos Nulos
Foi identificado a necessidade de tratamento de nulos nos datasets orders, products e order_reviews.  

* **Orders** foca no custumer_id e depois atualiza.
* **Products** fizemos o tratamento substituindo para 'outros' os produtos que estavam sem a categoria. Dimensões e peso nulos foram preenchidos com a média da categoria ou média geral
* **Order_reviews** os campos nulos removidos causariam grande impacto na analise pois interlacalam entre titulo do comentario e mensagem do comentario. Os campos, quando nulos, são preenchidos por 'sem titulo' e 'sem comentário'.


In [ ]:
def tratar_valores_nulos():
    print("Iniciando tratamento de valores nulos...")

    # orders
    print("Tratando nulos em 'orders'...")
    orders_df = globals()['orders'] # Acessa o DataFrame global
    orders_df.dropna(subset=['customer_id'], inplace=True)
    globals()['orders'] = orders_df # Garante que o global seja atualizado
    print("'orders' atualizado.")

    # products
    print("Tratando nulos em 'products'...")
    products_df = globals()['products']
    products_df['product_category_name'] = products_df['product_category_name'].fillna('outros')
    for col_dim in ['product_name_lenght', 'product_description_lenght', 'product_photos_qty', 'product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm']:
        if products_df[col_dim].isnull().any():
            # Tenta preencher com a média da categoria primeiro
            products_df[col_dim] = products_df.groupby('product_category_name')[col_dim].transform(lambda x: x.fillna(x.mean()))
            # Se ainda houver nulos (ex: categoria nova ou todos nulos na categoria), preenche com a média geral
            if products_df[col_dim].isnull().any():
                 products_df[col_dim] = products_df[col_dim].fillna(products_df[col_dim].mean())
    globals()['products'] = products_df
    print("'products' atualizado.")

    # order_reviews
    print("Tratando nulos em 'order_reviews'...")
    order_reviews_df = globals()['order_reviews']
    order_reviews_df['review_comment_title'] = order_reviews_df['review_comment_title'].fillna('sem_titulo')
    order_reviews_df['review_comment_message'] = order_reviews_df['review_comment_message'].fillna('sem_comentario')
    globals()['order_reviews'] = order_reviews_df
    print("'order_reviews' atualizado.")

    print("Tratamento de valores nulos concluído.")

# Chamar a função para tratar valores nulos
tratar_valores_nulos()

# Opcional: Verificar amostras
print("\n--- Amostra de DataFrames Após Tratamento de Nulos (Primeiras 3 Linhas) ---")
if 'orders' in globals():
    print("\nDataset: orders (após nulos)")
    print(globals()['orders'].head(3))
    print("--------------------")
if 'products' in globals():
    print("\nDataset: products (após nulos)")
    print(globals()['products'].head(3))
    print("--------------------")

print("\nExecutado")

##Remoção de duplicatas.

*   **Customers, sellers, products:** Duplicatas baseadas em seus respectivos IDs foram removidas. O registro original foi mantido e as demais entradas repetudas descartadase descartadas, como atualizações incorretas ou dados redundantes.
*   **order_items:** Esta aplicação remove registros duplicados da tabela de itens de pedido (order_items), usando uma chave composta com os campos: ID do pedido, ID do produto e ID do item no pedido. Quando existem linhas repetidas com esses mesmos valores, apenas a primeira ocorrência é mantida, e as demais são excluídas. Isso evita dados redundantes e garante que a base esteja limpa e consistente para análises, relatórios ou integrações com outros sistemas.
* **Geolocation:** Linhas inteiramente duplicadas foram removidas.





In [ ]:
#Preparação dos Dados - Remoção de Duplicatas
print("--- Preparação dos Dados - Remoção de Duplicatas ---")

# Remover duplicatas de 'customers'
print(f"Shape original de customers: {customers.shape}")
customers.drop_duplicates(subset=['customer_id'], keep='first', inplace=True)
print(f"Shape de customers após remover duplicatas: {customers.shape}")

# Remover duplicatas de 'sellers'
print(f"\nShape original de sellers: {sellers.shape}")
sellers.drop_duplicates(subset=['seller_id'], keep='first', inplace=True)
print(f"Shape de sellers após remover duplicatas: {sellers.shape}")

# Remover duplicatas de 'products'
# (products já foi modificado na seção de nulos, usamos a versão atual)
print(f"\nShape original de products: {products.shape}")
products.drop_duplicates(subset=['product_id'], keep='first', inplace=True)
print(f"Shape de products após remover duplicatas: {products.shape}")

# Remover duplicatas de 'order_items'
# (order_items já foi modificado na seção de conversão de tipos, usamos a versão atual)
print(f"\nShape original de order_items: {order_items.shape}")
order_items.drop_duplicates(subset=['order_id', 'product_id', 'order_item_id'], keep='first', inplace=True)
print(f"Shape de order_items após remover duplicatas: {order_items.shape}")

# Remover duplicatas de 'geolocation'
print(f"\nShape original de geolocation: {geolocation.shape}")
geolocation.drop_duplicates(inplace=True)
print(f"Shape de geolocation após remover duplicatas: {geolocation.shape}")

print("\nRemoção de duplicatas concluída.")
print("-----------------------------------------------------\n")

##Conversão dos Tipos de Dados
Nesta etapa, padronizamos os formatos de data nos datasets orders e order_items para viabilizar análises temporais precisas (como cálculo de prazos de entrega) e integração com outras bases. A função 'converter_tipos_dados' realiza as seguintes ações:

Conversão para Datetime:

*  No **dataset orders**, colunas como order_purchase_timestamp e
order_delivered_customer_date são convertidas para o tipo datetime
*  Em **order_items**, a coluna shipping_limit_date também é padronizada.


Verificação de Integridade: Exibe os tipos de dados antes e depois da conversão para garantir a correta transformação. Mostra uma amostra das primeiras linhas e o shape dos datasets após a alteração. Esta padronização é essencial para garantir consistência nas análises subsequentes.



In [ ]:
def converter_tipos_dados():
    print("Iniciando conversão de tipos de dados...")

    # orders DataFrame
    print("\nConvertendo colunas de data em 'orders'...")
    orders_df = globals()['orders']
    date_cols_orders = ['order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date']

    print("Tipos de dados ANTES da conversão em 'orders':")
    print(orders_df[date_cols_orders].dtypes)

    for col in date_cols_orders:
        orders_df[col] = pd.to_datetime(orders_df[col], errors='coerce')

    globals()['orders'] = orders_df
    print("\nTipos de dados DEPOIS da conversão em 'orders':")
    print(orders_df[date_cols_orders].dtypes)
    print(f"'orders' atualizado. Shape: {orders_df.shape}")

    # order_items DataFrame
    print("\nConvertendo colunas de data em 'order_items'...")
    order_items_df = globals()['order_items']
    date_cols_order_items = ['shipping_limit_date']

    print("Tipos de dados ANTES da conversão em 'order_items':")
    print(order_items_df[date_cols_order_items].dtypes)

    for col in date_cols_order_items:
        order_items_df[col] = pd.to_datetime(order_items_df[col], errors='coerce')

    globals()['order_items'] = order_items_df
    print("\nTipos de dados DEPOIS da conversão em 'order_items':")
    print(order_items_df[date_cols_order_items].dtypes)
    print(f"'order_items' atualizado. Shape: {order_items_df.shape}")

    print("\nConversão de tipos de dados concluída.")

# Chamar a função para converter os tipos de dados
converter_tipos_dados()

# Opcional: Verificar as primeiras linhas e dtypes para confirmar as mudanças
print("\n--- Amostra de DataFrames Após Conversão de Tipos (Primeiras 3 Linhas) ---")
if 'orders' in globals():
    print("\nDataset: orders (após conversão de tipos)")
    print(globals()['orders'][['order_id', 'order_purchase_timestamp', 'order_approved_at']].head(3))
    print(f"Shape: {globals()['orders'].shape}")
    print("--------------------")
if 'order_items' in globals():
    print("\nDataset: order_items (após conversão de tipos)")
    print(globals()['order_items'][['order_id', 'order_item_id', 'shipping_limit_date']].head(3))
    print(f"Shape: {globals()['order_items'].shape}")
    print("--------------------")

print("\nExecutado")

##Normalização das colunas
Nomes de categorias em 'products' e 'product_category_name_translation' foram convertidos para minúsculas, underscores substituídos por espaços e espaços extras removidos.

In [ ]:
# Preparação dos Dados - Normalização e Transformações Adicionais

# Normalização de colunas de texto
print("Normalizando colunas de texto...")

# Normalizar 'product_category_name' no DataFrame 'products'
if 'products' in globals():
    products_df = globals()['products']
    if 'product_category_name' in products_df.columns:
        products_df['product_category_name'] = products_df['product_category_name'].str.lower().str.replace('_', ' ', regex=False).str.strip()
        globals()['products'] = products_df
        print("Coluna 'product_category_name' em 'products' normalizada.")
        print("Exemplo após normalização em 'products':")
        print(products_df[['product_category_name']].head())
    else:
        print("Coluna 'product_category_name' não encontrada em 'products'.")
else:
    print("DataFrame 'products' não encontrado no escopo global.")

# Normalizar 'product_category_name' no DataFrame 'product_category_name_translation'
if 'product_category_name_translation' in globals():
    translation_df = globals()['product_category_name_translation']
    if 'product_category_name' in translation_df.columns:
        translation_df['product_category_name'] = translation_df['product_category_name'].str.lower().str.replace('_', ' ', regex=False).str.strip()
        globals()['product_category_name_translation'] = translation_df
        print("\nColuna 'product_category_name' em 'product_category_name_translation' normalizada.")
        print("Exemplo após normalização em 'product_category_name_translation':")
        print(translation_df[['product_category_name', 'product_category_name_english']].head())
    else:
        print("Coluna 'product_category_name' não encontrada em 'product_category_name_translation'.")
else:
    print("DataFrame 'product_category_name_translation' não encontrado no escopo global.")

print("-" * 50)

# Média da Geolocalização
Este código realiza a consolidação de dados geográficos, agregando as coordenadas por CEP para otimizar análises espaciais e integrações com outros datasets. O processo cria um novo DataFrame com as médias de latitude e longitude por prefixo de CEP, impactando diretamente a qualidade e eficiência das análises de distribuição geográfica.

Agregação de Coordenadas:
* Agrupa os dados do DataFrame geolocation por geolocation_zip_code_prefix
* Calcula a média das latitudes (geolocation_lat) e longitudes (geolocation_lng) para cada CEP

Verificações:
*   Confirma a existência do DataFrame geolocation
*   Valida a presença das colunas necessárias
*   Verifica se o DataFrame não está vazio



In [ ]:
# Agregação de Geolocalização
print("Agregando dados de geolocalização...")

if 'geolocation' in globals():
    geolocation_df = globals()['geolocation']

    # Verificar se o DataFrame não está vazio e se as colunas necessárias existem
    if not geolocation_df.empty and all(col in geolocation_df.columns for col in ['geolocation_zip_code_prefix', 'geolocation_lat', 'geolocation_lng']):
        geolocation_agg_df = geolocation_df.groupby('geolocation_zip_code_prefix').agg(
            geolocation_lat=('geolocation_lat', 'mean'),
            geolocation_lng=('geolocation_lng', 'mean')
        ).reset_index()

        globals()['geolocation_agg'] = geolocation_agg_df
        print("Agregação de geolocalização concluída. DataFrame 'geolocation_agg' criado.")
        print("Shape de 'geolocation_agg':", geolocation_agg_df.shape)
        print("Exemplo de 'geolocation_agg':")
        print(geolocation_agg_df.head())
    else:
        print("DataFrame 'geolocation' está vazio ou não contém as colunas necessárias para agregação.")
else:
    print("DataFrame 'geolocation' não encontrado no escopo global.")
print("-" * 50)

##Merge para Tradução das Categorias
Este código realiza a integração entre os DataFrames products e product_category_name_translation para criar uma coluna unificada de categorias de produtos (product_category_name_final), combinando os nomes originais com suas traduções em inglês quando disponíveis. Essa padronização gera impactos imediatos na qualidade dos dados, permitindo análises mais consistentes e integrações precisas com outros datasets.

Processo de Junção:

* Verifica a existência dos DataFrames products e product_category_name_translation
* Confirma a presença da coluna chave product_category_name em ambos
* Executa um merge left para preservar todos os produtos, mesmo sem tradução

Criação da Coluna Unificada:

* Prioriza a versão em inglês (product_category_name_english)
* Mantém o nome original quando a tradução não existe
* Atualiza globalmente o DataFrame products


In [ ]:
# Merge para tradução de categorias de produtos
print("Realizando merge da tradução de categorias de produtos...")

if 'products' in globals() and 'product_category_name_translation' in globals():
    products_df = globals()['products']
    translation_df = globals()['product_category_name_translation']

    # Verificar se as colunas de merge existem
    if 'product_category_name' in products_df.columns and 'product_category_name' in translation_df.columns:
        products_merged_df = pd.merge(
            products_df,
            translation_df,
            on='product_category_name',
            how='left'
        )

        # Criar 'product_category_name_final'
        # Usar 'product_category_name_english' se disponível, senão usar 'product_category_name' original
        products_merged_df['product_category_name_final'] = products_merged_df['product_category_name_english'].fillna(products_merged_df['product_category_name'])

        globals()['products'] = products_merged_df # Atualizar o DataFrame global 'products'

        print("Merge da tradução de categorias de produtos concluído.")
        print("DataFrame 'products' atualizado com 'product_category_name_final'.")
        print("Shape de 'products' após merge:", products_merged_df.shape)
        print("Exemplo de 'products' com a nova coluna (verifique 'product_category_name', 'product_category_name_english', 'product_category_name_final'):")
        print(products_merged_df[['product_category_name', 'product_category_name_english', 'product_category_name_final']].head())
    else:
        print("Coluna 'product_category_name' não encontrada em 'products' ou 'product_category_name_translation' para o merge.")
else:
    print("DataFrames 'products' ou 'product_category_name_translation' não encontrados no escopo global.")

print("-" * 50)
# Sumário da Preparação dos Dados
print("\n--- Descrição dos Passos de Preparação dos Dados ---")
print("1. Carregamento: Todos os arquivos CSV foram carregados em DataFrames pandas e disponibilizados globalmente (ajustado para caminhos 'data/').")
print("2. Conversão de Datas: Colunas de timestamp foram convertidas para o tipo datetime (com errors='coerce').")
print("3. Tratamento de Nulos:")
print("   - orders: Linhas com 'customer_id' nulo foram removidas.")
print("   - products: 'product_category_name' nulo foi preenchido com 'outros'.")
print("   - products: Dimensões e peso nulos foram preenchidos com a média da categoria ou média geral.")
print("   - order_reviews: 'review_comment_title' nulo preenchido com 'sem_titulo', 'review_comment_message' nulo preenchido com 'sem_comentario'.")
print("4. Remoção de Duplicatas:")
print("   - customers, sellers, products: Duplicatas baseadas em seus respectivos IDs foram removidas (keep='first').")
print("   - order_items: Duplicatas baseadas na chave composta (order_id, product_id, order_item_id) foram removidas (keep='first').")
print("   - geolocation: Linhas inteiramente duplicadas foram removidas.")
print("5. Normalização de Colunas de Texto:")
print("   - Nomes de categorias em 'products' e 'product_category_name_translation' foram convertidos para minúsculas, underscores substituídos por espaços e espaços extras removidos.")
print("6. Agregação de Geolocalização:")
print("   - Criado DataFrame 'geolocation_agg' com a média de latitude/longitude por 'geolocation_zip_code_prefix', disponível globalmente.")
print("7. Tradução de Categorias:")
print("   - DataFrame 'products' atualizado com nomes de categorias em inglês (coluna 'product_category_name_final'), usando a tradução ou o nome original.")
print("-----------------------------------------------------")
print("Limpeza e preparação dos dados concluídas.")


##Modelo Relacional e Cálculo Logístico
Primeiro, estabelece um modelo relacional completo, conectando os principais DataFrames do sistema através de chaves primárias e estrangeiras - como orders (pedidos) com customers (clientes), order_items (itens), order_payments (pagamentos) e order_reviews (avaliações), além de integrar informações de produtos, vendedores e geolocalização.

Segundo, desenvolveremos uma função Haversine robusta para cálculo preciso de distâncias geográficas, que inclui: conversão de coordenadas para radianos, aplicação da fórmula matemática considerando a curvatura da Terra, tratamento de valores inválidos (NaN) e um sistema sofisticado de arredondamento que analisa as terceira e quarta casas decimais para decidir sobre o arredondamento final.

Esta implementação permite não apenas análises relacionais completas entre todos os aspectos do negócio, mas também viabiliza cálculos logísticos precisos, como distâncias entre clientes e vendedores, otimização de rotas e análises de proximidade, fundamentais para estratégias de entrega e expansão territorial do e-commerce.

In [ ]:
#  Modelo Relacional e Funções Auxiliares

# O modelo relacional é implementado através de DataFrames e suas chaves.
# As conexões principais são:
# - orders.customer_id -> customers.customer_id
# - orders.order_id -> order_items.order_id
# - orders.order_id -> order_payments.order_id
# - orders.order_id -> order_reviews.order_id
# - order_items.product_id -> products.product_id
# - order_items.seller_id -> sellers.seller_id
# - customers.customer_zip_code_prefix -> geolocation_agg.geolocation_zip_code_prefix (para geolocalização do cliente)
# - sellers.seller_zip_code_prefix -> geolocation_agg.geolocation_zip_code_prefix (para geolocalização do vendedor)
# - products.product_category_name -> product_category_name_translation.product_category_name (para tradução)

# Cria o grafo
dot = Digraph(comment='Modelo Relacional Olist', format='png')
dot.attr(rankdir='LR', bgcolor='white')

# Estilo padrão das tabelas
table_style = {
    'shape': 'plaintext',
    'fontname': 'monospace',
}

# Define as tabelas (com PK e FK)
tabelas = {
    'orders': ['order_id (PK)', 'customer_id (FK)'],
    'customers': ['customer_id (PK)', 'zip_code_prefix (FK)'],
    'order_items': ['order_id (FK)', 'product_id (FK)', 'seller_id (FK)'],
    'order_payments': ['order_id (FK)'],
    'order_reviews': ['order_id (FK)'],
    'products': ['product_id (PK)', 'product_category_name (FK)'],
    'sellers': ['seller_id (PK)', 'zip_code_prefix (FK)'],
    'geolocation_agg': ['zip_code_prefix (PK)', 'lat', 'lng'],
    'product_category_name_translation': ['product_category_name (PK)', 'product_category_name_english'],
}

for tabela, colunas in tabelas.items():
    tabela_html = f"""<
    <table border="1" cellborder="0" cellspacing="0">
        <tr><td bgcolor="lightblue"><b>{tabela}</b></td></tr>""" + \
        "".join([f'<tr><td align="left">{col}</td></tr>' for col in colunas]) + \
        "</table>>"
    dot.node(tabela, tabela_html, **table_style)

# Define as relações (FK → PK)
relacoes = [
    ('orders', 'customers'),
    ('orders', 'order_items'),
    ('orders', 'order_payments'),
    ('orders', 'order_reviews'),
    ('order_items', 'products'),
    ('order_items', 'sellers'),
    ('customers', 'geolocation_agg'),
    ('sellers', 'geolocation_agg'),
    ('products', 'product_category_name_translation')
]


for origem, destino in relacoes:
    dot.edge(origem, destino)

display(dot)


###Haversine para Cálculo Logístico

In [ ]:
# Função Haversine para calcular distância entre duas coordenadas geográficas
def haversine(lat1, lon1, lat2, lon2):
    if pd.isna(lat1) or pd.isna(lon1) or pd.isna(lat2) or pd.isna(lon2):
        return np.nan
    R = 6371
    lat1_rad, lon1_rad, lat2_rad, lon2_rad = map(math.radians, [lat1, lon1, lat2, lon2])
    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad
    a = math.sin(dlat / 2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c

    if pd.isna(distance):
        return np.nan

    dist_times_10000 = distance * 10000

    int_dist_times_10000 = int(dist_times_10000)

    third_fourth_digits = int_dist_times_10000 % 100

    if third_fourth_digits > 50:
        rounded_distance = (int_dist_times_10000 // 100 + 1) / 100.0
    else:
        rounded_distance = (int_dist_times_10000 // 100) / 100.0

    return rounded_distance

print("Função haversine definida.")

# **Análise Exploratória de Dados**
Nesta seção, vamos explorar os dados para extrair insights e responder as seguintes perguntas de negócio.

a) Qual o volume de pedidos por mês? Existe sazonalidade nas vendas?

b) Qual a distribuição do tempo de entrega dos pedidos?

c) Qual a relação entre o valor do frete e a distância de entrega?

d) Quais são as categorias de produtos mais vendidas em termos de faturamento?

e) Quais estados brasileiros possuem o maior valor médio de pedido?

# Volume de Pedidos por Mês e Sazonalidade
Este código realiza uma análise temporal do volume de pedidos em um e-commerce, utilizando o DataFrame orders como base. Primeiro, filtra os registros para garantir que contenham datas válidas de compra (order_purchase_timestamp). Em seguida, cria uma coluna ano_mes para agrupar os pedidos por mês e ano, facilitando a análise mensal. O código agrupa os dados por período (ano_mes) e conta o número de pedidos (order_id), gerando um DataFrame resumido (volume_mensal_py) que exibe o volume de pedidos por mês.

# **Insigths**
O gráfico demonstra a sazonalidade nas vendas, com um padrão cíclico que se repete ao longo dos meses. **O pico mais pronunciado ocorre em novembro, coincidindo com a Black Friday**, onde o volume de pedidos atinge seu ápice anual, evidenciando o impacto das promoções nesta época do ano.

Paralelamente, observa-se uma tendência geral de **crescimento nas vendas ao longo do tempo, sugerindo uma expansão saudável do negócio**, embora com quedas pontuais em meses específicos que podem estar relacionados a fatores sazonais ou econômicos. Os vales entre os picos indicam períodos de menor atividade comercial, que merecem atenção para implementação de estratégias como promoções específicas ou ajustes de estoque.

A análise reforça a importância de monitorar esses ciclos para manter a competitividade e aproveitar ao máximo os períodos de alta demanda, como o caso da Black Friday, enquanto se desenvolvem ações para minimizar o impacto dos meses de menor movimento.

In [ ]:
df_py = orders.copy()
df_py = df_py[df_py['order_purchase_timestamp'].notna()]
df_py['ano_mes'] = df_py['order_purchase_timestamp'].dt.to_period('M')
volume_mensal_py = df_py.groupby('ano_mes')['order_id'].count().reset_index(name='numero_de_pedidos')
volume_mensal_py['ano_mes'] = volume_mensal_py['ano_mes'].astype(str)

print("\nVolume de Pedidos por Mês:")
print(volume_mensal_py.head())
print(f"\nTotal de meses analisados: {len(volume_mensal_py)}")
print(f"Período coberto: {volume_mensal_py['ano_mes'].min()} a {volume_mensal_py['ano_mes'].max()}")

print("\nAnálise de Sazonalidade:")
print("Observando os dados, podemos identificar picos e vales.")
print("Um pico notável ocorre geralmente em novembro (potencialmente devido à Black Friday).")

import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(12, 6))
sns.lineplot(data=volume_mensal_py, x='ano_mes', y='numero_de_pedidos')
plt.title('Volume de Pedidos por Mês')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

##Distribuição do Tempo de Entrega dos Pedidos
Este código realiza uma análise detalhada do tempo de entrega dos pedidos em um e-commerce, desde o momento da compra até a entrega ao cliente. O processo inclui:

Preparação dos dados:
*  Remoção de registros com datas ausentes de compra ou entrega
*  Cálculo do tempo de entrega em dias

Filtragem e análise:
*   Foco em entregas entre 0-100 dias para eliminar outliers extremos
*   Geração de estatísticas descritivas (média, mediana, percentis)
*   Visualização através de histograma com curva de densidade

# **Insights:**

A análise revela que, embora a maioria das entregas ocorra dentro do esperado (20-30 dias), existem casos que demandam atenção especial devido a prazos significativamente maiores, sugerindo oportunidades para otimização da cadeia logística.

*   Distribuição assimétrica à direita, com maioria das entregas em 20-30 dias
*   Presença de uma cauda longa indicando entregas excepcionalmente demoradas
*   A cauda longa da distribuição sugere que, embora menos frequentes, alguns pedidos podem levar significativamente mais tempo para serem entregues.


In [ ]:
print("\n--- 5.2 Distribuição do Tempo de Entrega dos Pedidos ---")

df_orders_py = orders.copy()

df_orders_py.dropna(subset=['order_purchase_timestamp', 'order_delivered_customer_date'], inplace=True)

df_orders_py['tempo_entrega_dias'] = (df_orders_py['order_delivered_customer_date'] - df_orders_py['order_purchase_timestamp']).dt.days

df_orders_py_filtrado = df_orders_py[(df_orders_py['tempo_entrega_dias'] >= 0) & (df_orders_py['tempo_entrega_dias'] <= 100)].copy() # Added .copy() to avoid SettingWithCopyWarning

print("\nDistribuição do Tempo de Entrega (em dias):")
print(df_orders_py_filtrado[['order_id', 'tempo_entrega_dias']].head())

print(f"\nDescrição do Tempo de Entrega (dias):\n{df_orders_py_filtrado['tempo_entrega_dias'].describe()}")

print("\nVisualizando a Distribuição do Tempo de Entrega:")
plt.figure(figsize=(12, 6))
sns.histplot(df_orders_py_filtrado['tempo_entrega_dias'], bins=50, kde=True)
plt.title('Distribuição do Tempo de Entrega dos Pedidos (0-100 dias)')
plt.xlabel('Tempo de Entrega (dias)')
plt.ylabel('Número de Pedidos')
plt.grid(True)
plt.show()

print("\nAnálise da Distribuição do Tempo de Entrega:")
print("A distribuição é assimétrica à direita, com a maioria das entregas ocorrendo em até ~20-30 dias.")
print("O filtro de 0-100 dias foi aplicado para melhor visualização e para remover outliers que poderiam distorcer a análise e o gráfico.")
print("A cauda longa da distribuição sugere que, embora menos frequentes, alguns pedidos podem levar significativamente mais tempo para serem entregues.")

##Relação entre o Valor do Frete e a Distância de Entrega.
Este código realiza uma análise completa entre o valor do frete e a distância de entrega em um e-commerce. O processo começa com a preparação dos dados, combinando informações de pedidos, clientes, vendedores e geolocalização através de múltiplos merges, tratando valores ausentes e conflitos de colunas. Em seguida, calcula as distâncias entre clientes e vendedores usando a função haversine, filtrando coordenadas inválidas.

A análise remove outliers extremos (fretes acima de R$300 e distâncias superiores a 4000 km) e calcula a correlação entre as variáveis. Os resultados são visualizados em um gráfico de dispersão claro, mostrando a relação entre distância e custo do frete.

**Tendência:** Se os pontos formam uma nuvem que se inclina para cima da esquerda para a direita, isso reforça a correlação positiva.
Dispersão: A dispersão dos pontos indica que outros fatores além da distância também influenciam o frete. Uma grande dispersão significa que, para uma mesma distância, pode haver uma variação considerável no valor do frete. Esses fatores podem incluir peso do produto, dimensões, tipo de serviço de entrega, urgência, etc.

**Outliers:** Mesmo após a filtragem, podem existir alguns pontos que se desviam significativamente do padrão geral, representando casos específicos.
Com base nos resultados típicos para este dataset, espera-se uma correlação positiva, mas não perfeitamente linear, indicando que a distância é um fator importante, mas não o único, na determinação do preço do frete.

In [ ]:
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("Preparando dados para análise de frete vs. distância...")

# Criar cópias para evitar modificar os DataFrames originais
df_items = order_items.copy()
df_orders = orders.copy()
df_customers = customers.copy()
df_sellers = sellers.copy()
df_geo_agg = geolocation_agg.copy()

#Merge para obter dados de cliente e vendedor no mesmo DataFrame
data_frete = pd.merge(df_items[['order_id', 'seller_id', 'freight_value']], df_orders[['order_id', 'customer_id']], on='order_id')
data_frete = pd.merge(data_frete, df_customers[['customer_id', 'customer_zip_code_prefix']], on='customer_id')
data_frete = pd.merge(data_frete, df_sellers[['seller_id', 'seller_zip_code_prefix']], on='seller_id')

#Merge com dados de geolocalização para clientes
data_frete = pd.merge(data_frete, df_geo_agg, left_on='customer_zip_code_prefix', right_on='geolocation_zip_code_prefix', how='left')
data_frete.rename(columns={'geolocation_lat': 'customer_lat', 'geolocation_lng': 'customer_lng'}, inplace=True)
# Remover a coluna de prefixo de CEP da geolocalização do cliente para evitar conflito no próximo merge
if 'geolocation_zip_code_prefix' in data_frete.columns:
    data_frete.drop(columns=['geolocation_zip_code_prefix'], inplace=True)

#Merge com dados de geolocalização para vendedores
data_frete = pd.merge(data_frete, df_geo_agg, left_on='seller_zip_code_prefix', right_on='geolocation_zip_code_prefix', how='left', suffixes=('_customer', '_seller'))
data_frete.rename(columns={'geolocation_lat': 'seller_lat',
                           'geolocation_lng': 'seller_lng',
                           'geolocation_zip_code_prefix': 'seller_geo_zip_code_prefix'}, inplace=True)

if 'geolocation_lat_seller' in data_frete.columns:
    data_frete.rename(columns={'geolocation_lat_seller': 'seller_lat',
                               'geolocation_lng_seller': 'seller_lng'}, inplace=True)
if 'seller_geo_zip_code_prefix' in data_frete.columns:
    data_frete.drop(columns=['seller_geo_zip_code_prefix'], inplace=True)


#Tratar valores ausentes e calcular distância
data_frete.dropna(subset=['customer_lat', 'customer_lng', 'seller_lat', 'seller_lng', 'freight_value'], inplace=True)

if data_frete.empty:
    print("Não foi possível obter dados suficientes (após merges e remoção de nulos) para calcular distâncias.")
else:
    print(f"Calculando distâncias para {len(data_frete)} itens...")
    # Aplicar a função haversine
    data_frete['distancia_km'] = data_frete.apply(
        lambda row: haversine(row['customer_lat'], row['customer_lng'], row['seller_lat'], row['seller_lng']),
        axis=1
    )
    print("Cálculo de distâncias concluído.")

    # Remover NaNs que podem surgir do cálculo de haversine
    data_frete.dropna(subset=['distancia_km'], inplace=True)

    # Filtrar outliers para melhor visualização e análise de correlação
    data_filtrada = data_frete[(data_frete['freight_value'] > 0) & (data_frete['freight_value'] < 300)]
    data_filtrada = data_filtrada[(data_filtrada['distancia_km'] > 0) & (data_filtrada['distancia_km'] < 4000)]

    if data_filtrada.empty:
        print("Nenhum dado após filtragem de outliers de frete/distância.")
    else:
        print("\nPrimeiras linhas dos dados de Frete e Distância (após tratamento arredondamento para cima quando .005+ ou mantendo o valor quando .004-e filtros):")
        print(data_filtrada[['order_id', 'freight_value', 'distancia_km']].head())

        #Calcular correlação
        correlacao = data_filtrada['distancia_km'].corr(data_filtrada['freight_value'])
        print(f"\nCorrelação entre distância (km) e valor do frete (R$): {correlacao:.2f}")

        #Visualizar a relação com um gráfico de dispersão
        plt.figure(figsize=(10, 6))
        sns.scatterplot(x='distancia_km', y='freight_value', data=data_filtrada.sample(min(1000, len(data_filtrada))), alpha=0.5) # Usar uma amostra para plots densos
        plt.title('Relação entre Distância de Entrega e Valor do Frete')
        plt.xlabel('Distância (km)')
        plt.ylabel('Valor do Frete (R$)')
        plt.show()



**Interpretação dos Resultados (Frete vs. Distância):**

A análise da correlação e o gráfico de dispersão nos ajudam a entender a relação entre a distância da entrega e o valor do frete.

*   **Correlação**: O valor da correlação (entre -1 e 1) indica a força e a direção da relação linear. Uma correlação positiva próxima de 1 sugere que, à medida que a distância aumenta, o valor do frete também tende a aumentar. Se for próxima de 0, a relação linear é fraca.
*   **Gráfico de Dispersão**: O gráfico mostra visualmente essa relação. Podemos observar:
    *   **Tendência**: Se os pontos formam uma nuvem que se inclina para cima da esquerda para a direita, isso reforça a correlação positiva.
    *   **Dispersão**: A dispersão dos pontos indica que outros fatores além da distância também influenciam o frete. Uma grande dispersão significa que, para uma mesma distância, pode haver uma variação considerável no valor do frete. Esses fatores podem incluir peso do produto, dimensões, tipo de serviço de entrega, urgência, etc.
    *   **Outliers**: Mesmo após a filtragem, podem existir alguns pontos que se desviam significativamente do padrão geral, representando casos específicos.

Com base nos resultados típicos para este dataset, espera-se uma correlação positiva, mas não perfeitamente linear, indicando que a distância é um fator importante, mas não o único, na determinação do preço do frete.

##Categorias de Produtos Mais Vendidas (Faturamento)
Este código realiza uma análise estratégica das categorias de produtos que mais contribuem para o faturamento de um e-commerce. O processo começa com a combinação dos dados de itens de pedidos com as informações de categorias de produtos, garantindo que apenas categorias válidas sejam consideradas (excluindo valores nulos ou marcados como 'outros')

Principais insights gerados:
*   As categorias que mais geram mais venda são: health_beauty, watches_gifts e
bed_bath_table
*   O faturamento de cada uma delas passa de 1 milhão de reais

Aplicações práticas:

*   Priorização de investimentos em estoque
*   Desenvolvimento de campanhas de marketing segmentadas
*   Identificação de oportunidades para expansão de catálogo ou remoção de nichos com poucas vendas

In [ ]:
print("--- Categorias de Produtos Mais Vendidas (Faturamento) ---")

#Copiar DataFrames para evitar modificar os originais
df_items_eda = order_items.copy()
df_products_eda = products.copy()

#Merge para obter a categoria do produto para cada item de pedido
merged_df_cat_fat = pd.merge(
    df_items_eda,
    df_products_eda[['product_id', 'product_category_name_final']],
    on='product_id',
    how='left'
)

#Remover linhas onde a categoria do produto é nula ou 'outros'
merged_df_cat_fat.dropna(subset=['product_category_name_final'], inplace=True)
merged_df_cat_fat = merged_df_cat_fat[merged_df_cat_fat['product_category_name_final'] != 'outros']

#Calcular o faturamento total por categoria
faturamento_por_categoria = merged_df_cat_fat.groupby('product_category_name_final')['price'].sum().reset_index()

#Ordenar por faturamento em ordem decrescente e pegar o top 10
top_10_categorias_faturamento = faturamento_por_categoria.sort_values(by='price', ascending=False).head(10)
top_10_categorias_faturamento.rename(columns={'product_category_name_final': 'categoria_produto', 'price': 'faturamento_total_R$'}, inplace=True)

#Formatar a coluna de faturamento como moeda brasileira
top_10_categorias_faturamento_display = top_10_categorias_faturamento.copy()
top_10_categorias_faturamento_display['faturamento_total_R$'] = top_10_categorias_faturamento_display['faturamento_total_R$'].map('R$ {:,.2f}'.format)


print("\nTop 10 Categorias de Produtos por Faturamento:")
print(top_10_categorias_faturamento_display)

# Visualização
plt.figure(figsize=(12, 8))
sns.barplot(
    x='faturamento_total_R$',
    y='categoria_produto',
    data=top_10_categorias_faturamento, # Usar o DataFrame para plotagem
    hue='categoria_produto',
    palette='viridis',
    legend=False
)
plt.title('Top 10 Categorias de Produtos por Faturamento', fontsize=16)
plt.xlabel('Faturamento Total (R$)', fontsize=14)
plt.ylabel('Categoria do Produto', fontsize=14)
plt.tight_layout()
plt.show()

print("\nAnálise das Categorias Mais Vendidas por Faturamento:")
print("O gráfico e a tabela acima mostram as 10 categorias de produtos que mais geraram receita.")
print("Essas informações são cruciais para estratégias de estoque, marketing e desenvolvimento de novos produtos.")

##Estados Brasileiros com Maior Valor Médio de Pedido
Este código realiza uma análise estratégica para identificar os estados brasileiros com os maiores valores médios de pedido em um e-commerce, combinando dados de pedidos, itens e clientes.

# **Insights:**
*   Os quatro estados com maior valor médio de compra são Paraíba (PB),  Amapá (AP), o Acre (AC) e a Alagoas (AL).
*   Identificação de padrões regionais de consumo. Os estamos com maior valor medio estão no Norte e Nordeste.
*   O valor do Top 10 parte de R$172 reais

Aplicações estratégicas:
*   Segmentação de campanhas de marketing como frete grátis
*   Ajuste de políticas comerciais e de frete
*   Análise de potencial de mercado por região




In [ ]:
print("--- Estados Brasileiros com Maior Valor Médio de Pedido ---")

# Copiar DataFrames para evitar modificar os originais
df_orders_eda = orders.copy()
df_items_eda = order_items.copy()
df_customers_eda = customers.copy()

# Calcular o valor total por pedido (soma dos preços dos itens)
valor_por_pedido = df_items_eda.groupby('order_id')['price'].sum().reset_index()
valor_por_pedido.rename(columns={'price': 'valor_total_pedido'}, inplace=True)

# Merge para adicionar customer_id ao valor do pedido
pedidos_com_valor = pd.merge(
    df_orders_eda[['order_id', 'customer_id']],
    valor_por_pedido,
    on='order_id',
    how='inner' # Considera apenas pedidos que têm itens
)

# Merge para adicionar o estado do cliente
pedidos_com_estado = pd.merge(
    pedidos_com_valor,
    df_customers_eda[['customer_id', 'customer_state']],
    on='customer_id',
    how='inner' # Considera apenas clientes que fizeram pedidos com itens
)

# Remover linhas com estado nulo, se houver
pedidos_com_estado.dropna(subset=['customer_state'], inplace=True)

# Calcula o valor médio do pedido por estado
valor_medio_por_estado = pedidos_com_estado.groupby('customer_state')['valor_total_pedido'].mean().reset_index()

# Ordena e pega os top 10 estados
top_10_estados_valor_medio = valor_medio_por_estado.sort_values(by='valor_total_pedido', ascending=False).head(10)
top_10_estados_valor_medio.rename(columns={'customer_state': 'estado', 'valor_total_pedido': 'valor_medio_pedido_R$'}, inplace=True)

# Formata a coluna de valor médio como moeda brasileira para exibição
top_10_estados_valor_medio_display = top_10_estados_valor_medio.copy()
top_10_estados_valor_medio_display['valor_medio_pedido_R$'] = top_10_estados_valor_medio_display['valor_medio_pedido_R$'].map('R$ {:,.2f}'.format)

print("\nTop 10 Estados por Valor Médio de Pedido:")
print(top_10_estados_valor_medio_display)

# Visualização
plt.figure(figsize=(12, 8))
sns.barplot(
    x='valor_medio_pedido_R$',
    y='estado',
    data=top_10_estados_valor_medio, # Usar o DataFrame original para plotagem
    hue='estado',
    palette='coolwarm',
    legend=False
)
plt.title('Top 10 Estados Brasileiros por Valor Médio de Pedido', fontsize=16)
plt.xlabel('Valor Médio do Pedido (R$)', fontsize=14)
plt.ylabel('Estado', fontsize=14)
plt.tight_layout()
plt.show()

print("\nAnálise do Valor Médio de Pedido por Estado:")
print("O gráfico e a tabela mostram os estados onde os clientes tendem a gastar mais por pedido, em média.")
print("Isso pode indicar maior poder aquisitivo, preferência por produtos mais caros, ou maior quantidade de itens por pedido nesses estados.")

#**Solução de Problemas de Negócio**

##Análise de Retenção
Este código realiza uma análise completa sobre a retenção de clientes em um e-commerce, combinando dados de pedidos e clientes para identificar padrões de compra recorrente. Com foco em clientes únicos (customer_unique_id) para evitar distorções e ter uma visualização clara do comportamento de compra.

# **Insights:**
*   Análise comparativa entre clientes únicos e recorrentes. Dos 96096 clientes únicos, 2997 (3.12%) realizaram mais de uma compra.
*   Uma taxa de retenção baixa pode indicar oportunidades para melhorar a fidelidade do cliente através de programas de lealdade, melhorias no serviço ou ofertas personalizadas.


Aplicações estratégicas:
*   Identificação de oportunidades para melhorar a fidelização
*   Análise de efetividade de estratégias de retenção já existentes
*   Segmentação de clientes para campanhas personalizadas


In [ ]:
print("--- Análise de Retenção de Clientes ---")

# Copiar DataFrames para evitar modificar os originais
df_orders_retention = orders.copy()
df_customers_retention = customers.copy()

# Merge orders com customers para obter customer_unique_id
# Seleciona apenas colunas necessárias
orders_subset = df_orders_retention[['order_id', 'customer_id', 'order_purchase_timestamp']]
customers_subset = df_customers_retention[['customer_id', 'customer_unique_id']]
merged_df_retention = pd.merge(orders_subset, customers_subset, on='customer_id', how='inner')

# Conta o número de pedidos por customer_unique_id
pedidos_por_cliente_unico = merged_df_retention.groupby('customer_unique_id')['order_id'].count().reset_index(name='numero_de_pedidos')

# Identifica clientes recorrentes (mais de 1 pedido)
clientes_recorrentes_df = pedidos_por_cliente_unico[pedidos_por_cliente_unico['numero_de_pedidos'] > 1]

numero_total_clientes_unicos = pedidos_por_cliente_unico['customer_unique_id'].nunique()
numero_clientes_recorrentes = clientes_recorrentes_df['customer_unique_id'].nunique()

taxa_retencao = 0
if numero_total_clientes_unicos > 0:
    taxa_retencao = (numero_clientes_recorrentes / numero_total_clientes_unicos) * 100

print(f"\nNúmero total de clientes únicos: {numero_total_clientes_unicos}")
print(f"Número de clientes recorrentes (mais de 1 pedido): {numero_clientes_recorrentes}")
print(f"Taxa de retenção de clientes: {taxa_retencao:.2f}%")

print("\nInsights sobre Retenção de Clientes:")
if taxa_retencao > 0:
    print(f"- Dos {numero_total_clientes_unicos} clientes únicos, {numero_clientes_recorrentes} ({taxa_retencao:.2f}%) realizaram mais de uma compra.")
    print("- Uma taxa de retenção baixa pode indicar oportunidades para melhorar a fidelidade do cliente através de programas de lealdade, melhorias no serviço ou ofertas personalizadas.")
else:
    print("- Não foi possível calcular a taxa de retenção ou não há clientes recorrentes no período analisado.")

# Análise da distribuição do número de pedidos por cliente
print("\nDistribuição do Número de Pedidos por Cliente:")
distribuicao_pedidos = pedidos_por_cliente_unico['numero_de_pedidos'].value_counts().sort_index()
print(distribuicao_pedidos)

# Visualização da distribuição do número de pedidos
plt.figure(figsize=(10, 6))
distribuicao_pedidos_plot = distribuicao_pedidos.head(10) # Limitar para melhor visualização se houver muitos valores
sns.barplot(x=distribuicao_pedidos_plot.index, y=distribuicao_pedidos_plot.values, palette='magma', hue=distribuicao_pedidos_plot.index, legend=False)
plt.title('Distribuição do Número de Pedidos por Cliente Único', fontsize=15)
plt.xlabel('Número de Pedidos Realizados', fontsize=12)
plt.ylabel('Número de Clientes', fontsize=12)
plt.xticks(rotation=45)
for index, value in enumerate(distribuicao_pedidos_plot.values):
    plt.text(index, value + 50, str(value), ha='center', va='bottom', fontsize=9)
plt.tight_layout()
plt.show()

print("\nAnálise da Distribuição:")
print("A grande maioria dos clientes fez apenas uma compra. Estratégias para incentivar uma segunda compra poderiam ter um impacto significativo.")

##Predição de Atraso na Entrega

Este código implementa um modelo de machine learning para prever atrasos na entrega dos pedidos. O processo inclui:

####1. Preparação dos Dados:

*   Relacionamento dos dados (pedidos, itens, produtos, vendedores, clientes);
*   Definição da variável alvo (atrasado) comparando datas de entrega real e estimada.

####2. Insigths;

####3. Performance medida pela acurácia no conjunto de teste;

####4. Análise de trade-off entre falsos positivos e falsos negativos;

####5. Identificação de quais features mais impactam as previsões;

####6. Aplicações Práticas;

####7. Alertas proativos para possíveis atrasos;

####8. Otimização logística com base nos fatores de risco identificados
####9. Melhoria na experiência do cliente através de comunicações preventivas



In [ ]:
print("\nPreparando dados para o modelo de predição de atraso...")

# Copiar DataFrames para evitar modificar os originais no escopo global desta célula
orders_df = orders.copy()
order_items_df = order_items.copy()
products_df = products.copy() # Deve conter 'product_category_name_final'
sellers_df = sellers.copy()
customers_df = customers.copy()

# Selecionar colunas relevantes de 'orders' e remover NaNs cruciais para a definição de 'atrasado' e para a feature 'tempo_estimado_entrega_dias'.
orders_subset = orders_df[['order_id', 'customer_id', 'order_purchase_timestamp', 'order_delivered_customer_date', 'order_estimated_delivery_date']].copy()
orders_subset.dropna(subset=['order_delivered_customer_date', 'order_estimated_delivery_date', 'order_purchase_timestamp'], inplace=True)
print(f"Shape de orders_subset após dropna inicial: {orders_subset.shape}")

# Merge orders_subset com order_items
data_modelo = pd.merge(orders_subset,
                       order_items_df[['order_id', 'product_id', 'seller_id', 'price', 'freight_value']],
                       on='order_id')
print(f"Shape após merge com order_items: {data_modelo.shape}")

# Merge data_modelo com products
# Assegura que 'product_category_name_final' exista em products.
if 'product_category_name_final' not in products_df.columns:
    print("Alerta: 'product_category_name_final' não encontrada em 'products_df'. Verifique o pré-processamento.")
    # Tentativa de criar a coluna se não existir, assumindo 'outros' como fallback.
    if 'product_category_name_english' in products_df.columns and 'product_category_name' in products_df.columns:
         products_df['product_category_name_final'] = products_df['product_category_name_english'].fillna(products_df['product_category_name'])
    elif 'product_category_name' in products_df.columns:
        products_df['product_category_name_final'] = products_df['product_category_name']
    else:
        products_df['product_category_name_final'] = 'outros'
    products_df['product_category_name_final'] = products_df['product_category_name_final'].fillna('outros')


data_modelo = pd.merge(data_modelo,
                       products_df[['product_id', 'product_category_name_final', 'product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm']],
                       on='product_id', how='left')
print(f"Shape após merge com products: {data_modelo.shape}")

# Merge data_modelo com sellers
data_modelo = pd.merge(data_modelo,
                       sellers_df[['seller_id', 'seller_zip_code_prefix', 'seller_city', 'seller_state']],
                       on='seller_id', how='left')
print(f"Shape após merge com sellers: {data_modelo.shape}")

# Merge data_modelo com customers
data_modelo = pd.merge(data_modelo,
                       customers_df[['customer_id', 'customer_zip_code_prefix', 'customer_city', 'customer_state']],
                       on='customer_id', how='left')
print(f"Shape após merge com customers: {data_modelo.shape}")

print("\nMerge dos DataFrames concluído.")

# --- Definir Variável Alvo (`atrasado`) ---
data_modelo['atrasado'] = (data_modelo['order_delivered_customer_date'] > data_modelo['order_estimated_delivery_date']).astype(int)
print(f"\nVariável alvo 'atrasado' definida. Distribuição:")
print(data_modelo['atrasado'].value_counts(normalize=True) * 100)

# --- Engenharia de Features ---
print("\nIniciando engenharia de features...")

# Tempo estimado de entrega em dias
data_modelo['tempo_estimado_entrega_dias'] = (data_modelo['order_estimated_delivery_date'] - data_modelo['order_purchase_timestamp']).dt.days

# Volume do produto (cm³)
# Tratar NaNs em dimensões antes de calcular o volume, preenchendo com 1
dim_cols = ['product_length_cm', 'product_height_cm', 'product_width_cm']
for col in dim_cols:
    if col in data_modelo.columns:
        data_modelo[col] = data_modelo[col].fillna(1)
    else:
        print(f"Alerta: Coluna de dimensão '{col}' não encontrada. O volume do produto pode não ser calculado corretamente.")
        data_modelo[col] = 1
data_modelo['product_volume_cm3'] = data_modelo['product_length_cm'] * data_modelo['product_height_cm'] * data_modelo['product_width_cm']

# Final de semana (Sábado=5, Domingo=6)
data_modelo['purchase_dayofweek'] = data_modelo['order_purchase_timestamp'].dt.dayofweek
data_modelo['is_weekend_purchase'] = data_modelo['purchase_dayofweek'].apply(lambda x: 1 if x >= 5 else 0)

# Entrega no mesmo estado?
# Tratar NaNs em seller_state e customer_state antes da comparação, preenchendo com 'desconhecido'
data_modelo['seller_state'] = data_modelo['seller_state'].fillna('desconhecido')
data_modelo['customer_state'] = data_modelo['customer_state'].fillna('desconhecido')
data_modelo['same_state_delivery'] = (data_modelo['seller_state'] == data_modelo['customer_state']).astype(int)

print("Engenharia de features concluída.")

# Seleciona features para o modelo e a variável alvo
features = [
    'price', 'freight_value', 'product_weight_g', 'product_volume_cm3',
    'tempo_estimado_entrega_dias', 'is_weekend_purchase', 'same_state_delivery',
    'product_category_name_final', 'seller_state', 'customer_state'
]
target = 'atrasado'

# Criar DataFrame final para o modelo e tratar NaNs nas features selecionadas (seguindo a lógica do script original)
data_modelo_final = data_modelo[features + [target]].copy()
print(f"\nShape antes de tratar NaNs nas features selecionadas para o modelo: {data_modelo_final.shape}")
print("Valores nulos ANTES do dropna específico para features do modelo:")
print(data_modelo_final[features].isnull().sum())

data_modelo_final.dropna(subset=features, inplace=True)

print(f"\nShape após tratar NaNs (dropna) nas features selecionadas: {data_modelo_final.shape}")
print("Valores nulos DEPOIS do dropna específico para features do modelo:")
print(data_modelo_final[features].isnull().sum())


if len(data_modelo_final) < 100:
    print("ALERTA: Dados insuficientes para treinar o modelo após preparação e remoção de NaNs.")
elif data_modelo_final[target].nunique() < 2:
    print(f"ALERTA: A variável alvo '{target}' tem menos de duas classes únicas após o pré-processamento. Não é possível treinar o modelo.")
else:
    print(f"Distribuição da variável alvo '{target}' em data_modelo_final (pronto para split):\n{data_modelo_final[target].value_counts(normalize=True)}")

# --- Dividir Dataset em Treino e Teste ---
if not data_modelo_final.empty and data_modelo_final[target].nunique() >= 2:
    X = data_modelo_final[features]
    y = data_modelo_final[target]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
    print(f"\nDivisão treino/teste concluída:")
    print(f"Tamanho do dataset de treino (X_train): {X_train.shape}")
    print(f"Tamanho do dataset de teste (X_test): {X_test.shape}")
    print(f"Distribuição da variável alvo no treino:\n{y_train.value_counts(normalize=True) * 100}")
    print(f"Distribuição da variável alvo no teste:\n{y_test.value_counts(normalize=True) * 100}")

    # --- Construir e Treinar Modelo ---
    print("\nConstruindo e treinando o modelo de Regressão Logística...")
    numeric_features_pipeline = ['price', 'freight_value', 'product_weight_g', 'product_volume_cm3', 'tempo_estimado_entrega_dias']
    categorical_features_pipeline = ['product_category_name_final', 'seller_state', 'customer_state']

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', Pipeline([('imputer', SimpleImputer(strategy='median')),
                               ('scaler', StandardScaler())]), numeric_features_pipeline),
            ('cat', Pipeline([('imputer', SimpleImputer(strategy='most_frequent')),
                               ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))]), categorical_features_pipeline) # sparse_output=False para facilitar a visualização das features transformadas, se necessário
        ],
        remainder='passthrough'
    )

    model_pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', LogisticRegression(random_state=42, solver='liblinear', max_iter=1000))
    ])

    model_pipeline.fit(X_train, y_train)
    print("Treinamento do modelo concluído.")

    # --- Avaliar Performance do Modelo ---
    print("\nAvaliando performance do modelo...")
    y_pred_train = model_pipeline.predict(X_train)
    y_pred_test = model_pipeline.predict(X_test)

    accuracy_train = accuracy_score(y_train, y_pred_train)
    accuracy_test = accuracy_score(y_test, y_pred_test)
    print(f"\nAcurácia no Treino: {accuracy_train:.4f}")
    print(f"Acurácia no Teste: {accuracy_test:.4f}")

    print("\nRelatório de Classificação (Teste):")
    target_names_report = ['Não Atrasado (0)', 'Atrasado (1)']
    print(classification_report(y_test, y_pred_test, target_names=target_names_report))

    print("\nMatriz de Confusão (Teste):")
    cm = confusion_matrix(y_test, y_pred_test)
    print(cm)

    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Previsto Não Atrasado', 'Previsto Atrasado'],
                yticklabels=['Real Não Atrasado', 'Real Atrasado'])
    plt.xlabel('Previsto')
    plt.ylabel('Real')
    plt.title('Matriz de Confusão (Teste)')
    plt.show()

    print("\n--- Interpretação dos Resultados ---")
    print(f"O modelo de Regressão Logística alcançou uma acurácia de {accuracy_test:.2%} no conjunto de teste.")
    print("Analise o Relatório de Classificação para entender a performance em cada classe.")
    print("  - Precisão (Precision) para 'Atrasado (1)': De todos os pedidos que o modelo previu como atrasados, qual percentual realmente atrasou?")
    print("  - Recall (Sensibilidade) para 'Atrasado (1)': De todos os pedidos que realmente atrasaram, qual percentual o modelo conseguiu identificar corretamente?")
    print("\nA Matriz de Confusão detalha os acertos e erros:")
    print(f"  - Verdadeiros Negativos (VN): {cm[0,0]} (Não atrasado, previsto como não atrasado)")
    print(f"  - Falsos Positivos (FP):      {cm[0,1]} (Não atrasado, previsto como atrasado)")
    print(f"  - Falsos Negativos (FN):      {cm[1,0]} (Atrasado, previsto como não atrasado)")
    print(f"  - Verdadeiros Positivos (VP): {cm[1,1]} (Atrasado, previsto como atrasado)")
    print("\nConsiderações:")
    print("- Se o recall para a classe 'Atrasado (1)' for baixo, o modelo pode estar falhando em identificar muitos dos pedidos que de fato atrasam.")
    print("- Melhorias podem incluir: engenharia de features mais avançada, outros algoritmos, ajuste de hiperparâmetros, tratamento de desbalanceamento de classes.")

else:
    print("\nALERTA FINAL: Modelo não treinado ou dados insuficientes/inválidos. Verifique as mensagens de alerta anteriores.")

print("\n--- Fim da Seção 6.2 ---")

##Segmentação de Clientes
Este código realiza uma análise RFM (Recência, Frequência e Valor Monetário) para segmentar clientes de um e-commerce em grupos com comportamentos similares. Primeiro, ele verifica se os DataFrames necessários (orders, order_items, order_payments) estão disponíveis. Em seguida, calcula os componentes RFM:

Recência: Dias desde a última compra (quanto menor, melhor).

Frequência: Número total de pedidos.

Valor Monetário: Total gasto pelo cliente.

Os dados são pré-processados (tratamento de valores nulos e escalonamento) e, usando o método K-Means, os clientes são agrupados em clusters (o número ideal é definido pelo "Método do Cotovelo"). Finalmente, o código gera visualizações dos clusters (gráficos de barras) e um resumo estatístico, permitindo identificar perfis como "Clientes Campeões" (compras recentes, frequentes e de alto valor) ou "Clientes em Risco" (inatividade recente).

In [ ]:
if 'orders' in globals() and 'order_items' in globals() and 'order_payments' in globals():
    orders_df = orders.copy()
    order_items_df = order_items.copy()
    order_payments_df = order_payments.copy()
    print("DataFrames 'orders', 'order_items' e 'order_payments' carregados e copiados para o escopo local da segmentação.")
else:
    print("ERRO: DataFrames 'orders', 'order_items' ou 'order_payments' não encontrados no escopo global. Carregue-os primeiro.")

# Cálculo dos Componentes RFM
print("Iniciando o cálculo dos componentes RFM...")

# Calcular Recência, Frequência e Valor Monetário (RFM)

# Data da última compra no dataset + 1 dia para ser o ponto de referência 'agora'
now = orders_df['order_purchase_timestamp'].max() + pd.Timedelta(days=1)
print(f"Data de referência para cálculo da Recência ('agora'): {now}")

# Agrupa por customer_id para calcular Recência e Frequência
rfm_df = orders_df.groupby('customer_id').agg(
    recencia=('order_purchase_timestamp', lambda x: (now - x.max()).days),
    frequencia=('order_id', 'count')
).reset_index()

# Calcula Valor Monetário
payments_agg = order_payments_df.groupby('order_id')['payment_value'].sum().reset_index()
order_payments_merged = pd.merge(orders_df[['order_id', 'customer_id']], payments_agg, on='order_id', how='left')
monetary_df = order_payments_merged.groupby('customer_id')['payment_value'].sum().reset_index()
monetary_df.rename(columns={'payment_value': 'valor_monetario'}, inplace=True)

# Junta os componentes R, F com M
rfm_df = pd.merge(rfm_df, monetary_df, on='customer_id', how='left')
rfm_df['valor_monetario'] = rfm_df['valor_monetario'].fillna(0)

print("\nDataFrame RFM criado:")
print(rfm_df.head())
print(f"\nDimensões do DataFrame RFM: {rfm_df.shape}")
print("\nEstatísticas descritivas do DataFrame RFM:")
print(rfm_df[['recencia', 'frequencia', 'valor_monetario']].describe())

# Análise Exploratória Inicial do RFM
### Análise Exploratória Inicial dos Dados RFM

# Visualização das Distribuições RFM
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
fig.suptitle('Distribuição dos Componentes RFM', fontsize=16)

sns.histplot(rfm_df['recencia'], kde=True, ax=axes[0], color='skyblue')
axes[0].set_title('Distribuição da Recência (dias)')
axes[0].set_xlabel('Recência (dias desde a última compra)')
axes[0].set_ylabel('Número de Clientes')

sns.histplot(rfm_df['frequencia'], kde=False, ax=axes[1], color='salmon', discrete=True)
axes[1].set_title('Distribuição da Frequência (nº de pedidos)')
axes[1].set_xlabel('Frequência (número de pedidos)')
axes[1].set_ylabel('Número de Clientes')
if rfm_df['frequencia'].max() < 20:
    axes[1].set_xticks(range(1, rfm_df['frequencia'].max() + 1))

sns.histplot(rfm_df['valor_monetario'], kde=True, ax=axes[2], color='lightgreen')
axes[2].set_title('Distribuição do Valor Monetário (R$)')
axes[2].set_xlabel('Valor Monetário Total Gasto (R$)')
axes[2].set_ylabel('Número de Clientes')

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

print("\nComentários sobre as distribuições:")
print("- Recência: Geralmente mostra uma cauda longa.")
print("- Frequência: Frequentemente, a maioria dos clientes tem apenas uma compra.")
print("- Valor Monetário: Também pode ser assimétrico.")

# Pré-processamento dos Dados RFM (Imputação e Escalonamento)
print("\nIniciando o pré-processamento dos dados RFM...")
rfm_features = rfm_df[['recencia', 'frequencia', 'valor_monetario']].copy()
imputer = SimpleImputer(strategy='mean')
rfm_imputed = imputer.fit_transform(rfm_features)
rfm_imputed_df = pd.DataFrame(rfm_imputed, columns=rfm_features.columns)
scaler = StandardScaler()
rfm_scaled = scaler.fit_transform(rfm_imputed_df)
rfm_scaled_df = pd.DataFrame(rfm_scaled, columns=rfm_features.columns)
print("\nDataFrame RFM escalado:")
print(rfm_scaled_df.head())
print("\nEstatísticas descritivas do DataFrame RFM escalado:")
print(rfm_scaled_df.describe())

# Determinando o Número Ótimo de Clusters (Método do Cotovelo - Elbow Method)
print("\nDeterminando o número ótimo de clusters usando o Método do Cotovelo...")
inertia = []
k_range = range(1, 11)
for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init='auto')
    kmeans.fit(rfm_scaled_df)
    inertia.append(kmeans.inertia_)

plt.figure(figsize=(10, 6))
plt.plot(k_range, inertia, marker='o', linestyle='-', color='blue')
plt.xlabel('Número de Clusters (k)')
plt.ylabel('Inércia')
plt.title('Método do Cotovelo para Determinar k Ótimo')
plt.xticks(k_range)
plt.grid(True)
plt.show()
print("\nInterpretação do Método do Cotovelo: Procure por um 'cotovelo' no gráfico.")

# Escolha do Número de Clusters
### Escolha do Número de Clusters (k)

# Aplicando K-Means Clustering
optimal_k = 4
print(f"\nNúmero ótimo de clusters escolhido: k = {optimal_k}")
print("Aplicando K-Means...")
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init='auto')
rfm_df['cluster'] = kmeans.fit_predict(rfm_scaled_df)
print("\nSegmentação de clientes concluída.")
print(rfm_df.head())
print("\nContagem de clientes por cluster:")
print(rfm_df['cluster'].value_counts().sort_index())

# Análise dos Segmentos de Clientes
print("\nAnalisando os segmentos de clientes...")
rfm_clusters_summary = rfm_df.groupby('cluster').agg(
    recencia_media=('recencia', 'mean'),
    frequencia_media=('frequencia', 'mean'),
    valor_monetario_medio=('valor_monetario', 'mean'),
    contagem_clientes=('customer_id', 'count')
).reset_index()
print("\nResumo dos clusters (médias RFM e contagem):")
print(rfm_clusters_summary)

# Visualização das características dos clusters
fig, axes = plt.subplots(1, 3, figsize=(18, 6))
fig.suptitle(f'Características Médias dos {optimal_k} Clusters RFM', fontsize=16)

# Recência Média por Cluster
sns.barplot(x='cluster', y='recencia_media', data=rfm_clusters_summary, ax=axes[0], hue='cluster', palette='coolwarm', legend=False) # Modificado
axes[0].set_title('Recência Média por Cluster')
axes[0].set_xlabel('Cluster')
axes[0].set_ylabel('Recência Média (dias)')

# Frequência Média por Cluster
sns.barplot(x='cluster', y='frequencia_media', data=rfm_clusters_summary, ax=axes[1], hue='cluster', palette='coolwarm', legend=False) # Modificado
axes[1].set_title('Frequência Média por Cluster')
axes[1].set_xlabel('Cluster')
axes[1].set_ylabel('Frequência Média (pedidos)')

# Valor Monetário Médio por Cluster
sns.barplot(x='cluster', y='valor_monetario_medio', data=rfm_clusters_summary, ax=axes[2], hue='cluster', palette='coolwarm', legend=False) # Modificado
axes[2].set_title('Valor Monetário Médio por Cluster')
axes[2].set_xlabel('Cluster')
axes[2].set_ylabel('Valor Monetário Médio (R$)')

plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

# Interpretação dos Segmentos
### Interpretação dos Segmentos de Clientes

#**Exemplo de Interpretação:**
#*   **Cluster 0 - Clientes Campeões/Fiéis:** Baixa Recência, Alta Frequência, Alto Valor Monetário.
#*   **Cluster 1 - Clientes em Risco/Precisam de Atenção:** Recência Média/Alta, Frequência Média/Baixa.
#*   **Cluster 2 - Novos Clientes/Potencial:** Recência Baixa/Média, Baixa Frequência.
#*   **Cluster 3 - Clientes Perdidos/Hibernando:** Alta Recência, Baixa Frequência, Baixo Valor Monetário.

##Análise de Satisfação
Este código analisa a satisfação de clientes de um e-commerce, cruzando dados de avaliações (review_score) com métricas de entrega, categorias de produtos e valores financeiros. Primeiro, ele prepara os dados, calculando o tempo real de entrega e a diferença em relação à estimativa. Em seguida, gera visualizações que mostram:

Distribuição das notas: A maioria dos clientes dá nota 5, mas há um número significativo de notas 1.

Relação entre notas e entrega: Notas baixas estão associadas a prazos mais longos e atrasos.

Desempenho por categoria: Identifica as categorias de produtos com melhores e piores avaliações médias.

Impacto de preço e frete: Verifica se valores muito altos ou baixos influenciam as notas.

In [ ]:
df_satisfacao = pd.merge(order_reviews, orders, on='order_id', how='left')

# Converter colunas de data para datetime
date_cols_satisfacao = ['order_purchase_timestamp', 'order_delivered_customer_date',
                        'order_estimated_delivery_date', 'review_creation_date',
                        'review_answer_timestamp']
for col in date_cols_satisfacao:
    if col in df_satisfacao.columns:
        df_satisfacao[col] = pd.to_datetime(df_satisfacao[col], errors='coerce')

# Calcular tempo de entrega real e diferença da estimativa
df_satisfacao['tempo_entrega_real_dias'] = (df_satisfacao['order_delivered_customer_date'] - df_satisfacao['order_purchase_timestamp']).dt.days
df_satisfacao['diferenca_estimativa_entrega_dias'] = (df_satisfacao['order_delivered_customer_date'] - df_satisfacao['order_estimated_delivery_date']).dt.days

print(f"Dados de satisfação preparados. Total de reviews: {len(df_satisfacao)}")
print(df_satisfacao[['order_id', 'review_score', 'tempo_entrega_real_dias', 'diferenca_estimativa_entrega_dias']].head())

# Análise da Pontuação de Avaliação (Review Score) ---
print("\nAnalisando a distribuição da Pontuação de Avaliação (Review Score)...")
plt.figure(figsize=(10, 6))
sns.countplot(x='review_score', data=df_satisfacao, palette='viridis', hue='review_score', legend=False)
plt.title('Distribuição da Pontuação de Avaliação (Review Score)', fontsize=15)
plt.xlabel('Review Score', fontsize=12)
plt.ylabel('Contagem', fontsize=12)
plt.grid(axis='y', linestyle='--')
plt.show()

print("\nInterpretação da Distribuição do Review Score:")
print("- A maioria dos clientes atribui a nota máxima (5 estrelas).")
print("- Há uma quantidade significativa de notas 1 estrela, indicando insatisfação em alguns casos.")
print("- Notas intermediárias (2, 3, 4) são menos comuns.")

# Relação entre Review Score e Métricas de Entrega ---
print("\nAnalisando Review Score vs. Métricas de Entrega...")
df_entrega_plot = df_satisfacao.dropna(subset=['review_score', 'tempo_entrega_real_dias', 'diferenca_estimativa_entrega_dias'])

if not df_entrega_plot.empty:
    fig, axes = plt.subplots(1, 2, figsize=(18, 7))
    fig.suptitle('Review Score vs. Métricas de Entrega', fontsize=16)

    # Tempo de Entrega Real vs. Review Score
    sns.boxplot(x='review_score', y='tempo_entrega_real_dias', data=df_entrega_plot, ax=axes[0], hue='review_score', palette='coolwarm', legend=False)
    axes[0].set_title('Tempo de Entrega Real (dias) por Review Score', fontsize=14)
    axes[0].set_xlabel('Review Score', fontsize=12)
    axes[0].set_ylabel('Tempo de Entrega Real (dias)', fontsize=12)
    if not df_entrega_plot.empty:
        axes[0].set_ylim(0, df_entrega_plot['tempo_entrega_real_dias'].quantile(0.95))

    # Diferença da Estimativa de Entrega vs. Review Score
    sns.boxplot(x='review_score', y='diferenca_estimativa_entrega_dias', data=df_entrega_plot, ax=axes[1], hue='review_score', palette='coolwarm', legend=False)
    axes[1].set_title('Diferença da Estimativa de Entrega (dias) por Review Score', fontsize=14)
    axes[1].set_xlabel('Review Score', fontsize=12)
    axes[1].set_ylabel('Diferença da Estimativa (dias) [Real - Estimada]', fontsize=12)
    if not df_entrega_plot.empty:
        lower_q = df_entrega_plot['diferenca_estimativa_entrega_dias'].quantile(0.05)
        upper_q = df_entrega_plot['diferenca_estimativa_entrega_dias'].quantile(0.95)
        axes[1].set_ylim(lower_q, upper_q)

    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

    print("\nInterpretação - Review Score vs. Métricas de Entrega:")
    print("- Tempo de Entrega: Notas mais baixas (1, 2) tendem a estar associadas a tempos de entrega mais longos.")
    print("- Diferença da Estimativa: Atrasos (valores positivos) estão fortemente correlacionados com notas baixas.")
else:
    print("Dados insuficientes para plotar Review Score vs. Métricas de Entrega.")

# --- Relação entre Review Score e Categoria do Produto ---
print("\nPreparando dados para análise de Review Score por Categoria de Produto...")
df_satisfacao_items = pd.merge(df_satisfacao, order_items[['order_id', 'product_id']], on='order_id', how='left')
df_satisfacao_items = pd.merge(df_satisfacao_items, products[['product_id', 'product_category_name']], on='product_id', how='left')
df_satisfacao_items = pd.merge(df_satisfacao_items, product_category_name_translation[['product_category_name', 'product_category_name_english']], on='product_category_name', how='left')
df_satisfacao_items.rename(columns={'product_category_name_english': 'categoria_produto_ingles'}, inplace=True)
df_satisfacao_items['categoria_produto_ingles'] = df_satisfacao_items['categoria_produto_ingles'].fillna('outros')

df_categoria_plot = df_satisfacao_items.dropna(subset=['review_score', 'categoria_produto_ingles'])

if not df_categoria_plot.empty and df_categoria_plot['categoria_produto_ingles'].nunique() > 1:
    media_score_categoria = df_categoria_plot.groupby('categoria_produto_ingles')['review_score'].agg(['mean', 'count']).sort_values(by='mean')

    # Filtrar categorias com poucas avaliações para uma análise mais estável
    media_score_categoria_filtrada = media_score_categoria[media_score_categoria['count'] >= 20] # Limite de contagem ajustável

    if not media_score_categoria_filtrada.empty:
        top_n_cat = min(7, len(media_score_categoria_filtrada))
        bottom_n_cat = min(7, len(media_score_categoria_filtrada))

        top_categorias = media_score_categoria_filtrada.tail(top_n_cat)
        bottom_categorias = media_score_categoria_filtrada.head(bottom_n_cat)

        print(f"\nTop {top_n_cat} Categorias por Média de Review Score (com >= 20 avaliações):")
        print(top_categorias)
        print(f"\nBottom {bottom_n_cat} Categorias por Média de Review Score (com >= 20 avaliações):")
        print(bottom_categorias)

        fig, axes = plt.subplots(2, 1, figsize=(14, 16))
        fig.suptitle('Review Score Médio por Categoria de Produto (Top e Bottom com >= 20 avaliações)', fontsize=16)

        sns.boxplot(x='review_score', y='categoria_produto_ingles',
                    data=df_categoria_plot[df_categoria_plot['categoria_produto_ingles'].isin(top_categorias.index)],
                    ax=axes[0], palette='summer', order=top_categorias.index.tolist()[::-1], hue='review_score', legend=False)
        axes[0].set_title(f'Top {top_n_cat} Categorias com Melhores Avaliações Médias', fontsize=14)
        axes[0].set_xlabel('Review Score', fontsize=12)
        axes[0].set_ylabel('Categoria do Produto (Inglês)', fontsize=12)

        sns.boxplot(x='review_score', y='categoria_produto_ingles',
                    data=df_categoria_plot[df_categoria_plot['categoria_produto_ingles'].isin(bottom_categorias.index)],
                    ax=axes[1], palette='autumn_r', order=bottom_categorias.index.tolist(), hue='review_score', legend=False)
        axes[1].set_title(f'Bottom {bottom_n_cat} Categorias com Piores Avaliações Médias', fontsize=14)
        axes[1].set_xlabel('Review Score', fontsize=12)
        axes[1].set_ylabel('Categoria do Produto (Inglês)', fontsize=12)

        plt.tight_layout(rect=[0, 0, 1, 0.95])
        plt.show()
        print("\nInterpretação - Review Score vs. Categoria: Certas categorias podem ter melhor ou pior avaliação média.")
    else:
        print("Poucas categorias com volume suficiente de avaliações para análise de Top/Bottom.")
else:
    print("Dados insuficientes para a análise de Review Score por Categoria de Produto.")

# --- Relação entre Review Score e Valores (Preço e Frete) ---
print("\nPreparando dados para análise de Review Score vs. Valores Financeiros...")
df_order_payment_totals = order_payments.groupby('order_id')['payment_value'].sum().reset_index()
df_order_payment_totals.rename(columns={'payment_value': 'total_price_order'}, inplace=True)

df_order_freight_totals = order_items.groupby('order_id')['freight_value'].sum().reset_index()
df_order_freight_totals.rename(columns={'freight_value': 'total_freight_order'}, inplace=True)

df_satisfacao_finance = pd.merge(df_satisfacao, df_order_payment_totals, on='order_id', how='left')
df_satisfacao_finance = pd.merge(df_satisfacao_finance, df_order_freight_totals, on='order_id', how='left')

df_finance_plot = df_satisfacao_finance.dropna(subset=['review_score', 'total_price_order', 'total_freight_order'])
df_finance_plot_filtered = pd.DataFrame() # Inicializa

if not df_finance_plot.empty:
    price_cap = df_finance_plot['total_price_order'].quantile(0.95)
    freight_cap = df_finance_plot['total_freight_order'].quantile(0.95)
    df_finance_plot_filtered = df_finance_plot[
        (df_finance_plot['total_price_order'] <= price_cap) & (df_finance_plot['total_price_order'] > 0) &
        (df_finance_plot['total_freight_order'] <= freight_cap) & (df_finance_plot['total_freight_order'] >= 0)
    ]

print("\nAnalisando Review Score vs. Preço Total e Frete Total do Pedido...")
if not df_finance_plot_filtered.empty:
    fig, axes = plt.subplots(1, 2, figsize=(18, 7))
    fig.suptitle('Review Score vs. Valores Financeiros do Pedido', fontsize=16)

    sns.boxplot(x='review_score', y='total_price_order', data=df_finance_plot_filtered, ax=axes[0], hue='review_score', palette='plasma_r', legend=False, showfliers=False)
    axes[0].set_title('Preço Total do Pedido por Review Score', fontsize=14)
    axes[0].set_xlabel('Review Score', fontsize=12)
    axes[0].set_ylabel('Preço Total do Pedido (R$)', fontsize=12)

    sns.boxplot(x='review_score', y='total_freight_order', data=df_finance_plot_filtered, ax=axes[1], hue='review_score', palette='plasma_r', legend=False, showfliers=False)
    axes[1].set_title('Frete Total do Pedido por Review Score', fontsize=14)
    axes[1].set_xlabel('Review Score', fontsize=12)
    axes[1].set_ylabel('Frete Total do Pedido (R$)', fontsize=12)

    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()
    print("\nInterpretação - Review Score vs. Valores: Analisar se preços ou fretes muito altos/baixos se correlacionam com notas específicas.")
else:
    print("Dados insuficientes para a análise de Review Score por Valores Financeiros após filtragem.")

print("\nAnálise de Satisfação do Cliente Concluída.")

#**Visualização e Dashboards**

# Dashboard geral de Evolução das Vendas

In [ ]:
# Configuração do estilo Seaborn
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('viridis')

# Função para formatar valores monetários
def format_currency(x, pos):
    return f'R$ {x:,.0f}'

# Preparar dados para o dashboard
print("Preparando dados para o dashboard de vendas...")

# Verificar se os DataFrames necessários estão disponíveis
required_dfs = ['orders', 'order_items', 'products', 'customers', 'product_category_name_translation']

all_dfs_loaded = True
for df_name in required_dfs:
    if df_name not in globals():
        print(f"Erro: DataFrame '{df_name}' não encontrado. Certifique-se de carregar todos os dados necessários.")
        all_dfs_loaded = False

if not all_dfs_loaded:
    print("Não é possível continuar com a criação do dashboard devido a DataFrames ausentes.")
else:
    # Cria DataFrame base para o dashboard
    dashboard_data = pd.merge(orders[['order_id', 'customer_id', 'order_purchase_timestamp']],
                              order_items[['order_id', 'product_id', 'price']],
                              on='order_id')
    dashboard_data = pd.merge(dashboard_data,
                              products[['product_id', 'product_category_name']],
                              on='product_id')
    dashboard_data = pd.merge(dashboard_data,
                              customers[['customer_id', 'customer_state']],
                              on='customer_id')
    dashboard_data = pd.merge(dashboard_data,
                              product_category_name_translation[['product_category_name', 'product_category_name_english']],
                              on='product_category_name',
                              how='left')

    # Tratamento de dados
    dashboard_data['product_category_name_english'] = dashboard_data['product_category_name_english'].fillna('undefined')
    dashboard_data['order_purchase_timestamp'] = pd.to_datetime(dashboard_data['order_purchase_timestamp'])
    dashboard_data['month_year'] = dashboard_data['order_purchase_timestamp'].dt.to_period('M')
    dashboard_data['month_year_formatted'] = dashboard_data['order_purchase_timestamp'].dt.strftime('%b/%Y')

    ALL_OPTION_TEXT = "Todos"

    states_list_unique = sorted(dashboard_data['customer_state'].unique())
    states_options = [ALL_OPTION_TEXT] + states_list_unique

    categories_list_unique = sorted(dashboard_data['product_category_name_english'].unique())
    categories_options = [ALL_OPTION_TEXT] + categories_list_unique

    # Criar widgets para filtros
    state_selector = widgets.SelectMultiple(
        options=states_options,
        value=[ALL_OPTION_TEXT],
        description='Estados:',
        disabled=False,
        layout=widgets.Layout(width='300px', height='150px')
    )

    category_selector = widgets.SelectMultiple(
        options=categories_options,
        value=[ALL_OPTION_TEXT],
        description='Categorias:',
        disabled=False,
        layout=widgets.Layout(width='300px', height='150px')
    )

    apply_button = widgets.Button(
        description='Aplicar Filtros',
        button_style='success',
        tooltip='Clique para aplicar os filtros selecionados'
    )

    # Container para os filtros
    filter_container = widgets.VBox([
        widgets.HTML(value="<h3>Filtros do Dashboard</h3>"),
        widgets.HBox([
            widgets.VBox([widgets.Label("Selecione os Estados:"), state_selector]),
            widgets.VBox([widgets.Label("Selecione as Categorias:"), category_selector])
        ]),
        apply_button
    ])


    dashboard_output = widgets.Output()

    fig_dashboard = None
    gs_dashboard = None
    kpi_area_dash, kpi_area2_dash, kpi_area3_dash = None, None, None
    ax1_dash, ax2_dash = None, None
    footer_ax_dash = None

    def create_dashboard_layout_once():
        global fig_dashboard, gs_dashboard, kpi_area_dash, kpi_area2_dash, kpi_area3_dash, ax1_dash, ax2_dash, footer_ax_dash
        if fig_dashboard is None:
            fig_dashboard = plt.figure(figsize=(18, 11), facecolor='white', constrained_layout=True)
            gs_dashboard = gridspec.GridSpec(4, 6, figure=fig_dashboard)

            kpi_area_dash = fig_dashboard.add_subplot(gs_dashboard[0, 0:2])
            kpi_area2_dash = fig_dashboard.add_subplot(gs_dashboard[0, 2:4])
            kpi_area3_dash = fig_dashboard.add_subplot(gs_dashboard[0, 4:6])
            ax1_dash = fig_dashboard.add_subplot(gs_dashboard[1:3, 0:3])
            ax2_dash = fig_dashboard.add_subplot(gs_dashboard[1:3, 3:6])
            footer_ax_dash = fig_dashboard.add_subplot(gs_dashboard[3, :])

    # Função para atualizar o dashboard com base nos filtros
    def update_dashboard_on_filter(b):
        global fig_dashboard, kpi_area_dash, kpi_area2_dash, kpi_area3_dash, ax1_dash, ax2_dash, footer_ax_dash

        if fig_dashboard is None:
            create_dashboard_layout_once()

        selected_states_raw = list(state_selector.value)
        selected_categories_raw = list(category_selector.value)

        apply_state_filter = bool(selected_states_raw) and ALL_OPTION_TEXT not in selected_states_raw
        actual_selected_states = selected_states_raw if apply_state_filter else []

        apply_category_filter = bool(selected_categories_raw) and ALL_OPTION_TEXT not in selected_categories_raw
        actual_selected_categories = selected_categories_raw if apply_category_filter else []

        # Aplicar filtros aos dados
        filtered_data = dashboard_data.copy()

        if apply_state_filter:
            filtered_data = filtered_data[filtered_data['customer_state'].isin(actual_selected_states)]

        if apply_category_filter:
            filtered_data = filtered_data[filtered_data['product_category_name_english'].isin(actual_selected_categories)]

        # Recalcular agregações
        monthly_filtered = filtered_data.groupby('month_year')['price'].sum().reset_index()
        monthly_filtered['month_year_str'] = monthly_filtered['month_year'].astype(str)

        category_filtered = filtered_data.groupby('product_category_name_english')['price'].sum().reset_index()
        category_filtered = category_filtered[category_filtered['product_category_name_english'] != 'undefined']
        category_filtered = category_filtered.sort_values('price', ascending=False).head(15).sort_values('price', ascending=True)


        # Calcular KPIs
        total_sales = filtered_data['price'].sum()
        avg_ticket = filtered_data.groupby('order_id')['price'].sum().mean() if not filtered_data.empty and 'order_id' in filtered_data else 0
        total_orders = filtered_data['order_id'].nunique()

        with dashboard_output:
            dashboard_output.clear_output(wait=True)
            all_axes = [kpi_area_dash, kpi_area2_dash, kpi_area3_dash, ax1_dash, ax2_dash, footer_ax_dash]
            for current_ax in all_axes:
                current_ax.clear()
                current_ax.grid(False)

            fig_dashboard.suptitle('DASHBOARD DE EVOLUÇÃO DE VENDAS', fontsize=20, fontweight='bold')

            kpi_area_dash.axis('off')
            kpi_area2_dash.axis('off')
            kpi_area3_dash.axis('off')
            footer_ax_dash.axis('off')

            kpi_area_dash.text(0.5, 0.5, f'Vendas Totais\n{format_currency(total_sales, None)}',
                         fontsize=14, fontweight='bold', ha='center', va='center',
                         bbox=dict(facecolor='lightblue', alpha=0.5, boxstyle='round,pad=0.5'))

            kpi_area2_dash.text(0.5, 0.5, f'Ticket Médio\n{format_currency(avg_ticket, None)}',
                          fontsize=14, fontweight='bold', ha='center', va='center',
                          bbox=dict(facecolor='lightgreen', alpha=0.5, boxstyle='round,pad=0.5'))

            kpi_area3_dash.text(0.5, 0.5, f'Total de Pedidos\n{total_orders:,}',
                          fontsize=14, fontweight='bold', ha='center', va='center',
                          bbox=dict(facecolor='lightsalmon', alpha=0.5, boxstyle='round,pad=0.5'))

            ax1_dash.set_title('Evolução Mensal de Vendas', fontsize=14, pad=10)
            ax1_dash.set_xlabel('Período', fontsize=12)
            ax1_dash.set_ylabel('Total de Vendas (R$)', fontsize=12)
            ax1_dash.yaxis.set_major_formatter(mtick.FuncFormatter(format_currency))
            if not monthly_filtered.empty:
                sns.lineplot(x='month_year_str', y='price', data=monthly_filtered, marker='o', ax=ax1_dash, linewidth=2, color=sns.color_palette('viridis')[0])
                ax1_dash.tick_params(axis='x', rotation=45)
                plt.setp(ax1_dash.get_xticklabels(), ha='right')
                for i, row in monthly_filtered.iterrows():
                    ax1_dash.annotate(f'{format_currency(row["price"], None)}',
                                (row['month_year_str'], row['price']),
                                textcoords="offset points", xytext=(0,10), ha='center', fontsize=8)
            else:
                ax1_dash.text(0.5, 0.5, 'Sem dados para o período selecionado', ha='center', va='center', fontsize=12)
            ax1_dash.grid(True, linestyle='--', alpha=0.7)

            ax2_dash.set_title('Top Categorias por Vendas', fontsize=14, pad=10)
            ax2_dash.set_xlabel('Total de Vendas (R$)', fontsize=12)
            ax2_dash.set_ylabel('Categoria de Produto', fontsize=12)
            ax2_dash.xaxis.set_major_formatter(mtick.FuncFormatter(format_currency))
            if not category_filtered.empty:
                bars = sns.barplot(x='price', y='product_category_name_english', data=category_filtered, ax=ax2_dash,
                                   palette='viridis', hue='product_category_name_english', dodge=False, legend=False)
                for i, p in enumerate(bars.patches):
                    width = p.get_width()
                    ax2_dash.text(width + (total_sales * 0.005 if total_sales else 10), # Dynamic offset
                                  p.get_y() + p.get_height()/2,
                                  f'{format_currency(width, None)}',
                                  ha='left', va='center', fontsize=8)
            else:
                ax2_dash.text(0.5, 0.5, 'Sem dados para as categorias selecionadas', ha='center', va='center', fontsize=12)
            ax2_dash.grid(True, axis='x', linestyle='--', alpha=0.7)

            filter_text_display = "Filtros aplicados: "
            if apply_state_filter:
                filter_text_display += f"Estados: {', '.join(actual_selected_states)} | "
            else:
                filter_text_display += "Todos os Estados | "

            if apply_category_filter:
                filter_text_display += f"Categorias: {', '.join(actual_selected_categories[:3])}"
                if len(actual_selected_categories) > 3:
                    filter_text_display += f" e mais {len(actual_selected_categories)-3}..."
            else:
                filter_text_display += "Todas as Categorias"

            min_date_str = dashboard_data['order_purchase_timestamp'].min().strftime('%d/%m/%Y') if not dashboard_data.empty else "N/A"
            max_date_str = dashboard_data['order_purchase_timestamp'].max().strftime('%d/%m/%Y') if not dashboard_data.empty else "N/A"
            date_range_text = f"Período dos dados: {min_date_str} a {max_date_str}"

            footer_ax_dash.text(0.01, 0.7, filter_text_display, fontsize=10, ha='left', va='center', wrap=True)
            footer_ax_dash.text(0.01, 0.4, date_range_text, fontsize=10, ha='left', va='center')
            footer_ax_dash.text(0.99, 0.1, f"Atualizado em: {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}", fontsize=9, ha='right', va='center', color='gray')


            display(fig_dashboard)


    if all_dfs_loaded:
        create_dashboard_layout_once()


        apply_button.on_click(update_dashboard_on_filter)


        display(filter_container)
        display(dashboard_output)

        update_dashboard_on_filter(None)

        print("\nDashboard de vendas interativo carregado. ")

Mapa de calor mostrando a concentração de vendas por região/estado do Brasil



In [ ]:
def gerar_mapa_densidade_clientes():
    """
    Gera um mapa de densidade de kernel (heatmap de pontos) mostrando a concentração
    geográfica de clientes no Brasil.
    Utiliza dados reais se 'customers' e 'geolocation_agg' estiverem disponíveis e mescláveis.
    Caso contrário, simula pontos aleatórios dentro dos limites do Brasil.
    """
    print("\n--- Gerando Mapa de Densidade de Clientes (Kernel Heatmap) ---")

    # Verificar se os DataFrames necessários estão carregados
    required_dfs_mapa = ['customers', 'geolocation_agg']
    dfs_disponiveis = True
    for df_name in required_dfs_mapa:
        if df_name not in globals():
            print(f"DataFrame '{df_name}' não encontrado globalmente. Necessário para o mapa de densidade com dados reais.")
            dfs_disponiveis = False
            break

    locations_data = None

    if dfs_disponiveis:
        print("Processando dados reais para o mapa de densidade...")
        customers_df = globals()['customers']
        geolocation_agg_df = globals()['geolocation_agg']

        if 'customer_zip_code_prefix' not in customers_df.columns:
            print("Erro: Coluna 'customer_zip_code_prefix' ausente no DataFrame 'customers'.")
            dfs_disponiveis = False
        if not all(col in geolocation_agg_df.columns for col in ['geolocation_zip_code_prefix', 'geolocation_lat', 'geolocation_lng']):
            print("Erro: Colunas 'geolocation_zip_code_prefix', 'geolocation_lat' ou 'geolocation_lng' ausentes no DataFrame 'geolocation_agg'.")
            dfs_disponiveis = False

        if dfs_disponiveis:
            # Juntar customers com geolocation_agg para obter lat/lng
            merged_geo_customers = pd.merge(
                customers_df[['customer_id', 'customer_zip_code_prefix']],
                geolocation_agg_df[['geolocation_zip_code_prefix', 'geolocation_lat', 'geolocation_lng']],
                left_on='customer_zip_code_prefix',
                right_on='geolocation_zip_code_prefix',
                how='inner'
            )
            merged_geo_customers.dropna(subset=['geolocation_lat', 'geolocation_lng'], inplace=True)

            if not merged_geo_customers.empty:
                locations_data = merged_geo_customers[['geolocation_lat', 'geolocation_lng']].values.tolist()
                print(f"Dados de localização de {len(locations_data)} clientes preparados.")
            else:
                print("Nenhum dado de localização de cliente encontrado após o merge. Verifique os dados de CEP e geolocalização.")
                dfs_disponiveis = False
        else:
             print("Não foi possível prosseguir com dados reais devido a colunas ausentes.")


    if not dfs_disponiveis or locations_data is None or len(locations_data) == 0:
        print("Não é possível gerar o mapa de densidade com dados reais. Simulando pontos no Brasil...")
        min_lat, max_lat = -33.75, 5.27
        min_lon, max_lon = -73.98, -34.79
        np.random.seed(42)
        sim_lat = np.random.uniform(min_lat, max_lat)
        sim_lon = np.random.uniform(min_lon, max_lon)
        locations_data = [[lat, lon] for lat, lon in zip(sim_lat, sim_lon)]


    if locations_data is None or len(locations_data) == 0:
        print("Nenhum dado de localização (real ou simulado) disponível. Não é possível gerar o mapa.")
        return

    map_center_lat = -14.2350
    map_center_lon = -51.9253
    brazil_map = folium.Map(location=[map_center_lat, map_center_lon], zoom_start=4)

    HeatMap(locations_data, radius=10, blur=10).add_to(brazil_map)

    print("Gerando o mapa de densidade...")
    display(brazil_map)
    print("Mapa de densidade de clientes gerado e exibido.")


gerar_mapa_densidade_clientes()

# Impacto do Tempo de Entrega na Avaliação do Cliente

In [ ]:
def visualizar_impacto_tempo_entrega_na_avaliacao():
    """
    Gera gráficos para visualizar como o tempo de entrega impacta a avaliação dos clientes.
    1. Gráfico de Barras: Avaliação média por faixas de tempo de entrega.
    2. Boxplot: Distribuição do tempo de entrega para cada nota de avaliação.
    """
    print("\n--- Visualizando Impacto do Tempo de Entrega na Avaliação do Cliente ---")

    if 'orders' not in globals() or 'order_reviews' not in globals():
        print("DataFrames 'orders' ou 'order_reviews' não encontrados globalmente.")
        print("Por favor, carregue e prepare os dados em células anteriores antes de executar esta.")
        return

    reviews_df = globals()['order_reviews'][['order_id', 'review_score']].copy()
    orders_df = globals()['orders'][['order_id', 'order_purchase_timestamp', 'order_delivered_customer_date']].copy()

    df_viz = pd.merge(reviews_df, orders_df, on='order_id', how='left')

    df_viz['order_purchase_timestamp'] = pd.to_datetime(df_viz['order_purchase_timestamp'])
    df_viz['order_delivered_customer_date'] = pd.to_datetime(df_viz['order_delivered_customer_date'])
    df_viz['tempo_entrega_dias'] = (df_viz['order_delivered_customer_date'] - df_viz['order_purchase_timestamp']).dt.days

    df_viz.dropna(subset=['review_score', 'tempo_entrega_dias'], inplace=True)
    df_viz = df_viz[(df_viz['tempo_entrega_dias'] >= 0) & (df_viz['tempo_entrega_dias'] <= 90)]

    if df_viz.empty:
        print("Nenhum dado disponível após a preparação para visualização do impacto do tempo de entrega.")
        return

    fig, axes = plt.subplots(2, 1, figsize=(14, 16))
    fig.suptitle('Impacto do Tempo de Entrega na Avaliação do Cliente', fontsize=18, y=1.03)

    max_days_for_bins = int(df_viz['tempo_entrega_dias'].max())
    if max_days_for_bins == 0:
        max_days_for_bins = 1

    if max_days_for_bins <= 10:
        bins = list(range(0, max_days_for_bins + 2, 1))
        labels = [f'{i} dia(s)' for i in range(len(bins)-1)]
    elif max_days_for_bins <= 30:
        bins = list(range(0, max_days_for_bins + 4, 3))
        labels = [f'{bins[i]}-{bins[i+1]-1} dias' for i in range(len(bins)-1)]
    else:
        bins = [0, 8, 16, 31, 46, 61, max_days_for_bins + 1]
        labels = ['0-7 dias', '8-15 dias', '16-30 dias', '31-45 dias', '46-60 dias', f'61-{max_days_for_bins} dias']

    bins = sorted(list(set(bins)))
    if bins[-1] <= max_days_for_bins:
        bins[-1] = max_days_for_bins + 1

    current_labels = []
    for i in range(len(bins)-1):
        if bins[i+1] == bins[i]+1:
             current_labels.append(f'{bins[i]} dia(s)')
        else:
             current_labels.append(f'{bins[i]}-{bins[i+1]-1} dias')
    labels = current_labels

    df_viz['faixa_tempo_entrega'] = pd.cut(df_viz['tempo_entrega_dias'],
                                           bins=bins,
                                           labels=labels,
                                           right=False,
                                           include_lowest=True)

    avg_score_by_delivery_time = df_viz.groupby('faixa_tempo_entrega', observed=False)['review_score'].mean().reset_index()

    sns.barplot(x='faixa_tempo_entrega', y='review_score', data=avg_score_by_delivery_time, ax=axes[0], palette="coolwarm", hue='faixa_tempo_entrega', dodge=False, legend=False)
    axes[0].set_title('Avaliação Média do Cliente por Faixa de Tempo de Entrega', fontsize=14)
    axes[0].set_xlabel('Tempo de Entrega', fontsize=12)
    axes[0].set_ylabel('Avaliação Média (1-5)', fontsize=12)
    axes[0].set_ylim(0, 5.5)
    axes[0].tick_params(axis='x', rotation=45)
    plt.setp(axes[0].get_xticklabels(), ha='right', rotation_mode='anchor') # Alinhar os rótulos

    for index, row in avg_score_by_delivery_time.iterrows():
        if pd.notnull(row.review_score):
            axes[0].text(index, row.review_score + 0.1, f'{row.review_score:.2f}', color='black', ha="center", fontsize=9)

    sns.boxplot(x='review_score', y='tempo_entrega_dias', data=df_viz, ax=axes[1], palette="viridis", hue='review_score', legend=False)
    axes[1].set_title('Distribuição do Tempo de Entrega por Nota de Avaliação do Cliente', fontsize=14)
    axes[1].set_xlabel('Avaliação do Cliente (1-5)', fontsize=12)
    axes[1].set_ylabel('Tempo de Entrega (dias)', fontsize=12)

    plt.tight_layout(rect=[0, 0, 1, 0.97])
    plt.show()

    print("Visualizações do impacto do tempo de entrega na avaliação geradas.")

# Chamada da função para gerar os gráficos.
visualizar_impacto_tempo_entrega_na_avaliacao()

# Dashboard de Análise de Vendedores
Um dashboard de análise dos vendedores, mostrando quais têm melhor desempenho em termos de volume de vendas, satisfação do cliente e tempo de entrega

In [ ]:
def dashboard_analise_vendedores():
    """
    Gera um dashboard de análise de desempenho dos vendedores, mostrando
    volume de vendas, satisfação do cliente e tempo de entrega.
    """
    print("\n--- Dashboard de Análise de Vendedores ---")

    required_dfs = ['order_items', 'orders', 'order_reviews', 'sellers']
    if not all(df_name in globals() for df_name in required_dfs):
        print(f"DataFrames necessários ({', '.join(required_dfs)}) não encontrados globalmente.")
        print("Por favor, carregue e prepare os dados em células anteriores antes de executar esta.")
        return

    # Usa cópias para evitar modificar os DataFrames globais
    df_items = globals()['order_items'].copy()
    df_orders = globals()['orders'].copy()
    df_reviews = globals()['order_reviews'].copy()
    df_sellers = globals()['sellers'][['seller_id', 'seller_city', 'seller_state']].copy()

    # 1. Preparar dados base
    df_merged = pd.merge(df_items[['order_id', 'order_item_id', 'product_id', 'seller_id', 'price', 'freight_value']],
                         df_orders[['order_id', 'customer_id', 'order_purchase_timestamp',
                                    'order_delivered_customer_date', 'order_status']],
                         on='order_id', how='inner')

    df_merged = pd.merge(df_merged,
                         df_reviews[['order_id', 'review_score']],
                         on='order_id', how='left')

    df_merged['order_purchase_timestamp'] = pd.to_datetime(df_merged['order_purchase_timestamp'])
    df_merged['order_delivered_customer_date'] = pd.to_datetime(df_merged['order_delivered_customer_date'])
    df_merged['tempo_entrega_dias'] = (df_merged['order_delivered_customer_date'] - df_merged['order_purchase_timestamp']).dt.days

    df_entregues = df_merged[df_merged['order_status'] == 'delivered'].copy()
    df_entregues.dropna(subset=['tempo_entrega_dias'], inplace=True)
    df_entregues = df_entregues[df_entregues['tempo_entrega_dias'] >= 0]

    # 2. Calcular métricas por vendedor
    vendas_por_vendedor = df_merged.groupby('seller_id').agg(
        total_vendas_valor=('price', 'sum'),
        total_itens_vendidos=('order_item_id', 'count')
    ).reset_index()

    satisfacao_por_vendedor = df_entregues.dropna(subset=['review_score']).groupby('seller_id').agg(
        media_review_score=('review_score', 'mean'),
        num_reviews=('review_score', 'count')
    ).reset_index()

    entrega_por_vendedor = df_entregues.groupby('seller_id').agg(
        tempo_medio_entrega_dias=('tempo_entrega_dias', 'mean'),
        num_entregas=('order_id', 'nunique')
    ).reset_index()

    df_vendedor_performance = pd.merge(vendas_por_vendedor, satisfacao_por_vendedor, on='seller_id', how='left')
    df_vendedor_performance = pd.merge(df_vendedor_performance, entrega_por_vendedor, on='seller_id', how='left')
    df_vendedor_performance = pd.merge(df_vendedor_performance, df_sellers, on='seller_id', how='left')

    if df_vendedor_performance.empty:
        print("Não foi possível gerar dados de performance dos vendedores.")
        return

    print("\nDataFrame de Performance dos Vendedores (Top 5 por Vendas):")
    display(df_vendedor_performance.sort_values(by='total_vendas_valor', ascending=False).head())

    # 3. Visualizações
    fig, axes = plt.subplots(3, 1, figsize=(14, 20)) # Ajustado figsize para melhor visualização no Colab
    fig.suptitle('Dashboard de Análise de Desempenho dos Vendedores', fontsize=18, y=1.01)

    top_n = 10
    min_reviews_threshold = 10
    min_entregas_threshold = 10

    # Gráfico 1: Top N Vendedores por Volume de Vendas
    top_vendedores_volume = df_vendedor_performance.sort_values(by='total_vendas_valor', ascending=False).head(top_n)
    sns.barplot(x='total_vendas_valor', y='seller_id', data=top_vendedores_volume, ax=axes[0], palette='viridis', hue='seller_id', dodge=False, legend=False)
    axes[0].set_title(f'Top {top_n} Vendedores por Volume de Vendas (R$)', fontsize=15)
    axes[0].set_xlabel('Volume de Vendas (R$)', fontsize=13)
    axes[0].set_ylabel('ID do Vendedor', fontsize=13)
    axes[0].tick_params(axis='y', rotation=0)
    for i, p in enumerate(axes[0].patches):
        value = p.get_width()
        axes[0].text(value + (top_vendedores_volume['total_vendas_valor'].max() * 0.01),
                     p.get_y() + p.get_height() / 2,
                     f'R$ {value:,.0f}', va='center', fontsize=10)

    # Gráfico 2: Top N Vendedores por Satisfação Média do Cliente
    vendedores_com_reviews_suficientes = df_vendedor_performance[df_vendedor_performance['num_reviews'] >= min_reviews_threshold]
    if not vendedores_com_reviews_suficientes.empty:
        top_vendedores_satisfacao = vendedores_com_reviews_suficientes.sort_values(by='media_review_score', ascending=False).head(top_n)
        sns.barplot(x='media_review_score', y='seller_id', data=top_vendedores_satisfacao, ax=axes[1], palette='coolwarm', hue='seller_id', dodge=False, legend=False)
        axes[1].set_title(f'Top {top_n} Vendedores por Satisfação Média (Mín. {min_reviews_threshold} Reviews)', fontsize=15)
        axes[1].set_xlabel('Satisfação Média (1-5)', fontsize=13)
        axes[1].set_ylabel('ID do Vendedor', fontsize=13)
        axes[1].set_xlim(0, 5.5)
        axes[1].tick_params(axis='y', rotation=0)
        for i, p in enumerate(axes[1].patches):
            value = p.get_width()
            axes[1].text(value + 0.05, p.get_y() + p.get_height() / 2, f'{value:.2f}', va='center', fontsize=10)
    else:
        axes[1].text(0.5, 0.5, f'Não há vendedores suficientes com >={min_reviews_threshold} reviews.', ha='center', va='center', fontsize=12, transform=axes[1].transAxes)
        axes[1].set_title(f'Top {top_n} Vendedores por Satisfação Média', fontsize=15)

    # Gráfico 3: Top N Vendedores por Menor Tempo Médio de Entrega
    vendedores_com_entregas_suficientes = df_vendedor_performance[df_vendedor_performance['num_entregas'] >= min_entregas_threshold]
    if not vendedores_com_entregas_suficientes.empty:
        top_vendedores_entrega = vendedores_com_entregas_suficientes.sort_values(by='tempo_medio_entrega_dias', ascending=True).head(top_n)
        sns.barplot(x='tempo_medio_entrega_dias', y='seller_id', data=top_vendedores_entrega, ax=axes[2], palette='mako', hue='seller_id', dodge=False, legend=False)
        axes[2].set_title(f'Top {top_n} Vendedores por Menor Tempo Médio de Entrega (Mín. {min_entregas_threshold} Entregas)', fontsize=15)
        axes[2].set_xlabel('Tempo Médio de Entrega (dias)', fontsize=13)
        axes[2].set_ylabel('ID do Vendedor', fontsize=13)
        axes[2].tick_params(axis='y', rotation=0)
        for i, p in enumerate(axes[2].patches):
            value = p.get_width()
            axes[2].text(value + (top_vendedores_entrega['tempo_medio_entrega_dias'].max() * 0.01 if not top_vendedores_entrega.empty else 0.1),
                         p.get_y() + p.get_height() / 2,
                         f'{value:.1f} dias', va='center', fontsize=10)
    else:
        axes[2].text(0.5, 0.5, f'Não há vendedores suficientes com >={min_entregas_threshold} entregas.', ha='center', va='center', fontsize=12, transform=axes[2].transAxes)
        axes[2].set_title(f'Top {top_n} Vendedores por Menor Tempo Médio de Entrega', fontsize=15)

    plt.tight_layout(rect=[0, 0, 1, 0.98])
    plt.show()

    print("Dashboard de análise de vendedores gerado.")

dashboard_analise_vendedores()